In [1]:
import pandas as pd

In [2]:
# 곡 정보 파일 불러오기
df = pd.read_csv('../SongTidy/FinalTidy/tidydata/song_tidy03.csv')

In [4]:
# 가수 정보 파일 불러오기
df2 = pd.read_csv('../Data/Data20180921/artist_info_combined_ver04.csv', encoding='euc-kr')

In [5]:
# 곡 정보와 가수 정보를 머지해준다.
song = pd.merge(df, df2, on='artist')

In [6]:
# 가사 분석에 용이하도록 <br>을 공백으로 바꿔준다.
song['lyrics'] = song['lyrics'].str.replace(r'<br>', ' ')

In [9]:
song.shape

(9566, 20)

In [7]:
# 타이틀 곡은 몇곡일까
len(song[song.is_title == '타이틀 곡'])

2379

In [8]:
# !pip install soynlp
# !pip show soynlp

In [9]:
# !pip install gensim
# !pip show gensim

In [33]:
import numpy as np
import re

In [34]:
from soynlp.tokenizer import RegexTokenizer
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus


tokenizer = RegexTokenizer()
tokenizer

In [12]:
female = song[song['gender'] == '여']

In [13]:
male = song[song['gender'] == '남']
# 남자아이돌 전체 곡 수
male.shape

(5869, 20)

In [14]:
# 여자아이돌 곡 중 제목에 '오빠' (혹은 오빤)가 등장하는 곡 수는?
female_o = female[female['title'].str.contains(r"(오빠|오빤)")]
female_o.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(4, 20)

In [16]:
# 남자아이돌 곡 중 제목에 '누나' (혹은 누난)가 등장하는 곡 수는?
male_o = male[male['title'].str.contains(r"(누나|누난)")]
male_o.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(4, 20)

In [22]:
# 여자아이돌 곡 중 가사에 '오빠' (혹은 오빤)가 등장하는 곡 수는?
female_o = female[female['lyrics'].str.contains(r"(오빠|오빤)", na=False)]
female_o.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(51, 20)

In [24]:
# 남자아이돌 곡 중 제목에 '누나' (혹은 누난)가 등장하는 곡 수는?
male_n = male[male['lyrics'].str.contains(r"(누나|누난)")]
male_n.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(30, 20)

In [26]:
# 여자아이돌 곡 중 가사에 '누나' (혹은 누난)가 등장하는 곡 수는?
female_n = female[female['lyrics'].str.contains(r"(누나|누난)", na=False)]
female_n.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(12, 20)

In [27]:
# 남자아이돌 곡 중 가사에 '오빠' (혹은 오빤)가 등장하는 곡 수는?
male_n = male[male['lyrics'].str.contains(r"(오빠|오빤)")]
male_n.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(49, 20)

In [20]:
# 전체 곡 중 가사에 '예쁘다' ([이예][쁘쁜뻐뻤])가 등장하는 곡 수는?
song_p = song[song['lyrics'].str.contains("[이예][쁘쁜뻐뻤]")]
song_p.shape

(1051, 20)

In [19]:
# 전체 곡 중 가사에 '잘생기다'(잘생) 가 등장하는 곡 수는?
song_h = song[song['lyrics'].str.contains(r"잘생")]
song_h.shape

(28, 20)

In [28]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [29]:
def tokenize(df):
    sentences = df['lyrics'].apply(preprocessing)
    tokens = sentences.apply(tokenizer.tokenize)
    tokens = tokens.tolist()
    tokens_all = sum(tokens, [])
    return tokens_all

In [35]:
female_t = tokenize(female)
male_t = tokenize(male)
song_t = tokenize(song)

In [25]:
song_t = song_t.copy()
female_t = female_t.copy()
male_t = male_t.copy()

In [26]:
# 토크나이즈한 단어들의 갯수
len(song_t)

2244770

# 예쁜과 잘생긴을 빼고 가사의 길이를 봐 보자 (토크나이즈한 단어의 갯수)

In [42]:
def preprocessing_p(text):
    # 개행문자 제거
#     text =  text.strip('\t\n\r')
#     pattern = re.compile(r'\s+')
#     text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[이예][쁘쁜뻐뻤]', '', text)
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
#     text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [43]:
preprocessing_p

TypeError: expected string or bytes-like object

In [ ]:
.strip

In [27]:
# 두 음절 이상으로 묶어준다.
# more_words = []
# for v in result2:
#     if len(v) > 1:
#          more_words.append(v)

NameError: name 'result2' is not defined

In [131]:
# unique(next_word(song_t, '여자는'))

In [132]:
# unique(next_word(song_t, '남자는'))

In [109]:
# search_word 앞에 무슨 단어가 있는지 보는 함수

def before_word(tokens, pattern):
    indices = [i for i, x in enumerate(tokens) if re.match(pattern, x) is not None]
    result = [tokens[i - 5:i + 2] for i in indices]
    return result

In [ ]:
# search_word 뒤에 무슨 단어가 있는지 보는 함수

def next_word(tokens, pattern):
    indices = [i for i, x in enumerate(tokens) if re.match(pattern, x) is not None]
    result = [tokens[i - 2:i + 5] for i in indices]
    return result

In [108]:
# search_word 앞, 뒤에 무슨 단어가 있는지 보는 함수

def before_next_word(tokens, pattern):
    indices = [i for i, x in enumerate(tokens) if re.match(pattern, x) is not None]
    result = [tokens[i - 5:i + 5] for i in indices]
    return result

In [34]:
# 중복되는 구절이 있어서 만든 유니크한 값만 보는 함수
def unique(word):
    result = []
    for i in word:
        if i not in result: #inside for
            result.append(i) #inside for inside if
    return result
    print(result)

In [ ]:
# 가장 많이 나온 값 보는 함수
def common(list):
    list = sum(list, [])  # 하나의 리스트로 만들어주기
    result = Counter(list).most_common(30)
    return result

In [137]:
unique(before_next_word(song_t, r'치마'))

[['good', 'mood', '필요', '없어', '비싼', '치마', '높은', '구두', '필요', '없어'],
 ['띄어', '한눈에', '알아봤어', '검은', '짧은', '치마', 'highheel', '이', '위태로워', '보인걸'],
 ['Oh', 'Oh', '나랑', '밀당하지마', '짧은', '치마를', '입고', '널', '만나러', '가고'],
 ['지쳐가', '짙은', '화장을', '하고', '짧은', '치마를', '입으면', '뜨거운', '시선을', '느낄'],
 ['내', '다릴', '봐', '예쁘잖아', '짧은', '치마', '입었잖아', '관심있게', '보란말야', 'oh'],
 ['내', '다릴', '봐', '예쁘잖아', '짧은', '치마', '입었잖아', '집에', '가기', '싫어질걸'],
 ['아이', '엠', '그라운드', '자기소개', '하기', '치마보단', '난', '긴', '청바지', '근데도'],
 ['내가', '하이힐', '립스틱', '그리고', '짧은', '치마', '하나', '둘씩', '변해가', '알'],
 ['away', 'Give', 'it', 'away', 'now', '치마', '좀', '내려', '딱', '달라'],
 ['지금껏', '불편해서', '자주', '입지', '않던', '치마', '이제는', '네게', '잘', '보이려'],
 ['안했어', '오늘따라', '유난히', '더', '예뻐', '치마', '입었네', '불편하다며', '못보던', '옷인데'],
 ['show', 'time', '노란', '생머리', '핫핑크색', '치마', '잘', '어울릴', '것', '같지'],
 ['나오려해', '나도', '날', '모르기에', '짧아진', '치마', '빨간', '립스틱', '풀어', '해쳐진'],
 ['나는', '고마워', '뾰족하고', '흰구두와', '어색한', '치마정장', '그', '당시', '누구나하던', '일진표'],
 ['한숨을', '크게', '쉬면', '날이', '밝아와', 

In [142]:
unique(before_next_word(song_t, r'바지'))

[['반지는', '언제든', '낄', '수', '있게', '바지', '주머니', '안에', '넌', '왜'],
 ['like', '피카소', '파도가', '날', '덮쳐', '바지가', '축축해져도', 'That', 's', 'Ok'],
 ['봤다면', '버섯', '먹고', '크게', '자라', '바지', '위에', '팬티를', '입고', '망토를'],
 ['그렇게', '놀랐니', '토끼', '토끼', '줄행랑', '바지는', '입고가', '웃기', '웃기지', '말고'],
 ['수', '없이', '하지만', '너는', '내게', '바지만', '입는다고', '투정', '부렸었지', '아파'],
 ['가고파', '난', 'Yeah', 'uh', '촌스러운', '바지', '엄마가', '입혀주는', '대로', '입던'],
 ['지구를', '부탁하노라', '아버지', '걱정은', '하지마세요', '바지위에', '팬티입고', '오늘도', '난', '길을'],
 ['몇몇은', '아마', '오줌을', '지렸어', '너', '바지', '짚업이', '안되', '삐져서', '나온'],
 ['천천히', 'Slowly', '그러다', '체하지', '니', '바지', '먹은', '특정', '부위', '스키니바지'],
 ['마요', '잘못돼요', '너', '꽉', '낀', '바지', '긴', '생머리', '했다고', '좀'],
 ['사람', '날', '부르는', '이름', '찢어진', '바지를', '입고', '튀는', '머리가', '사람들'],
 ['안경에', '다', '늘어난', '듯한', '추리닝', '바지까지도', '아', '뭐', '이렇게', '예뻐'],
 ['서울', '올라왔지', '난', '똥', '싼', '바지', '정상을', '원하지만', '현실은', '바닥'],
 ['death', 'of', 'art', '살짝', '내린', '바지', 'snapback', '이', '필수', '목에'],
 ['키커', '목소리는', '얇지만', '안', '잠기는', '바지지퍼', '날

In [135]:
unique(before_next_word(song_t, r'(수트|suit)'))

[['위에', '팬티를', '입고', '망토를', '달아', '수트도', '가끔', '입고', '가슴에서', '빛이'],
 ['I', 'll', 'be', 'your', 'sweet', 'suite', 'room', 'Baby', 'take', 'a'],
 ['몰라', '난', '그래', '마지막야', '멋진', 'suit', '를', '차려입고', '널', '닮은'],
 ['맘이', '원한', '걸', '쫓아', '무거운', '수트', '벗어', '벗어', '모차르트', '헨델'],
 ['이곳이',
  '오늘밤과',
  '나의',
  '거래처',
  'Black',
  'suit',
  'Black',
  'shoes',
  'Black',
  'tie'],
 ['러브', '도도도도', '도도해', 'Hey', '거기', '수트', '입은', '샹하이', 'Boy', '왜'],
 ['본드', '캐주얼', '한', '옷', '조차', '수트', '빨', '간지', '밤', '잠들지']]

In [139]:
unique(before_next_word(song_t, r'(명품|핸드)백'))

[['까칠하진', '않아', '뻐끔거리는', '금붕어', 'No', '명품백', '안에', '든', '사랑', 'No'],
 ['서로', '같이', '어딜', '가든', '내', '핸드백은', '니', '허리', 'Yo', 'ma'],
 ['필요할', '거야', '니', '손에', '들린', '핸드백이', '살짝', '내', '다릴', '스치고'],
 ['그녀', '곁에서라면', '무엇이든', '되도', '좋아요', '핸드백', '손수건과', '따뜻한', '코트', '립스틱'],
 ['그녀', '곁에서라면', '무엇이든', '돼도', '좋아요', '핸드백', '손수건과', '따뜻한', '코트', '립스틱'],
 ['promis', 'e', '변함없는', '가치', '비싼', '명품백', '다', '가질', '것만', '같지'],
 ['Baby', '저', '테이블', '아래로', '그녀', '핸드백', '속에도', '가득', '넘쳐', 'Yeah'],
 ['보단', '운동화가', '난', '좋아', '불편한', '핸드백', '보단', '편한', '가방이', '좋아'],
 ['눈물쯤', '얼마든지', '부딪혀도', '괜찮아', '엣지있는', '핸드백을', '메고', '제일', '아끼는', '예쁜']]

In [140]:
unique(before_next_word(song_t, r'가방'))

[['때', '내가', '아님', '어울리지', '않는', '가방과', '구둘', '봤을', '때', 'It'],
 ['대관이', '아찌', '마구', '울부짖었잖아', '이렇게', '가방', '끌어안고', '있는', '날', '보고'],
 ['좋아하는', '것', '이것저것', '챙겨', '늘어나는', '가방이', '좀', '무거워도', '난', '참을'],
 ['하루', '여기까지만', 'All', 'Stop', '작은', '가방', '운동화', '챙겨', '자', '더'],
 ['낡고', '먼지', '쌓인', '작고', '작은', '가방', '하나', '후후', '불어', '오래된'],
 ['못해', '넌', '내가', '필요해', '비싼', '가방과', '멋진', 'Fancy', 'Car', '난'],
 ['바쁘게', '가나요', '몸보다', '큰', '빨간', '가방', '메고', '힘들지는', '않나요', '휴'],
 ['오래', '전에', '쓴', '편지는', '아직도', '가방', '한', '켠에', '사실은', '말야'],
 ['것만', '보지', '멍청한', '어린', '꼰대들', '가방끈이', '짧아', '부', '니가', '귀염을'],
 ['재수없고', '오늘', '숙제역시', '모르고', '텅빈', '가방', '텅빈', '머리', '삔으로', '꽂아'],
 ['카페', '니가', '앉던', '자리에', '무심히', '가방을', '놓네', '모든', '게', '그대로인데'],
 ['위로', '들어', '길', '가던', '학생들도', '가방을', '집어던지고', '다', '같이', 'Shake'],
 ['기다릴까', '새로운', '무언가', '기타', '하나', '가방을', '메고', '모든', '걸', '잊고'],
 ['내', '나라가', '맞나', '예쁜', '교복을', '가방에', '넣고', '밤거리로', '찾아', '헤매는'],
 ['신고', '바퀴', '쌩쌩거리며', '왼손엔', '실내화', '가방을', '질질', '끌며', '뛰

In [141]:
unique(before_next_word(song_t, r'다리'))

[['지금', '우리', '다음', '세기로', '갈', '다리', '위', '그', '다리', '무너지기'],
 ['세기로', '갈', '다리', '위', '그', '다리', '무너지기', '저', '위기', '근데'],
 ['그러니', '내말', '들어봐', '등을', '기대고', '다리를', '쭉', '뻗어봐', '그리고', '달콤한'],
 ['멀리', '간다고', '말을', '했다는', '거야', '다리가', '풀려서', '주저', '앉고', '나서'],
 ['다른', '사람', '있었는지', '어쩜', '머리부터', '다리', '끝까지', '다른', '사람이', '있을까'],
 ['넌', '머리부터', '허리', 'to', 'the', '다리까지', '다', '그래', '난', '너를'],
 ['level', 'up', 'that', '내', '팔과', '다리는', '오프로드용', '사륜구동', '거친', '도로의'],
 ['스타일에', '너의', '헤어스타일에', '마네킹', '같은', '다리', 'Skinny', 'black', 'pants', '한쪽'],
 ['Woo', 'woo', 'woo', 'woo', '내', '다리를', '힐끔', '쳐다보네', 'Woo', 'woo'],
 ['안될', '것', '같아', '왜', '자꾸', '다리에', '힘이', '풀리나', '아스팔드에', '발이'],
 ['oh', 'oh', 'oh', '눈', '말고', '다리를', '봐', '손을', '놓고', '나를'],
 ['여러분들', '정말', '많이', '기다리시는데', '재중씨', '다리는', '많이', '나으셨나요', '아', '네'],
 ['많이', '나으셨나요', '아', '네', '어', '다리가', '너무', '많이', '좋아져서', '지금은'],
 ['딱', '멀리서', '네가', '보인다', '부서진', '다리', '위에서', '휘청이며', '휘청이며', '넌'],
 ['눈', '코', '입', '표정도', '팔', '다리', '걸음도', '

In [127]:
# 두 음절 이상의 단어중에서 빈도수 상위 30 번째까지 출력 

topn = 50 

def more_word(word):     
    more_words = []
    for v1 in word:
        for v2 in v1:
             if len(v2) > 1:
                more_words.append(v2)
    result = Counter(more_words).most_common(topn)
    return result

In [128]:
# both_words_ratio(female_back_words, male_back_words, female_back_len, male_back_len)

def both_words_ratio(f_, m_, f_count, m_count):
       
    f = pd.DataFrame(f_)
    m = pd.DataFrame(m_)
    f.columns = ['words', 'count_f']
    m.columns = ['words', 'count_m']

# (1) 두 그룹 모두에게서 등장한 단어들의 비율을 보는 함수

    merged_in = pd.merge(f, m, how='inner', on=['words'])    
    f['count_f'] = (merged_in['count_f']/f_count)*100
    m['count_m'] = (merged_in['count_m']/m_count)*100
    
    f = f.dropna()
    m = m.dropna()
    result = pd.merge(f, m, how='inner', on=['words'])  
    
# (2) 한 그룹에서만 등장한 단어들의 비율을 보는 함수    

    merged_out = pd.merge(f, m, how='outer', on=['words'])    
    f['count_f'] = (merged_in['count_f']/f_count)*100
    m['count_m'] = (merged_in['count_m']/m_count)*100
    result = pd.merge(f, m, how='outer', on=['words']) 

    return result

In [94]:
wannabe = before_word(female_t, r'(사람|사람[은이을])')
unique(wannabe)

[['아는지', '확신이', '없어', '너란', '사람', '그냥'],
 ['말고', '싸우지', '말고', '그', '사람에게', '잘해줘'],
 ['Get', 'Out', '너', '같은', '사람', '될까'],
 ['사람', '될까', '봐', '너라는', '사람이', '싫어'],
 ['늦지', '않았어', '사랑은', '한', '사람을', '바라보는'],
 ['늦지', '않았어', '사랑은', '한', '사람만', '바라보는'],
 ['살래요', '슬픔따위', '없을거라', '약속했던', '사람', '그대'],
 ['누굴', '만나도', '그대', '같은', '사람', '없어요'],
 ['you', '더운', '여름', '짜증난', '사람들', '날'],
 ['알고', '있잖아', '기다리고', '있는', '사람', '누군지'],
 ['나', '같은', '나', '같은', '사람', '다운'],
 ['나', '같은', '사람', '다운', '사람', '만나'],
 ['이제는', '이제는', '너와', '다른', '사람', '만나'],
 ['나', '같은', '나', '같은', '사람다운', '사람'],
 ['같은', '나', '같은', '사람다운', '사람', '만나'],
 ['있고', '용서로', '인해', '그', '사람', '또한'],
 ['그려져', 'Um', '하나에서', '너란', '사람을', '더해서'],
 ['사는', '이유', '하나에서', '너란', '사람을', '더해서'],
 ['어리숙하고', '난', '낯을', '가리는데', '사람을', '내'],
 ['입술이', '말라가', '꼭', '다른', '사람', '같은'],
 ['가더라도', '내', '편이', '돼줄', '사람', '어디에'],
 ['어디에', '있더라도', '나만', '생각할', '사람', '이게'],
 ['좋아', '넌', '그저', '남자', '사람', '친구'],
 ['소원은', '늘', '같아요', '한', '사람만', '바랬죠'],
 ['순간', '모두',

In [42]:
wannabe = before_word(male_t, '되고')
unique(wannabe)

[['가슴속에', '밀려오는', '잔잔한', '파도가', '되고', '싶어'],
 ['넌', '나에게', '잊지못할', '기억이', '되고', '싶어'],
 ['난', '너에게', '소중했던', '추억이', '되고', '싶어'],
 ['밝히니', '촛불이', '두', '개가', '되고', '그'],
 ['또', '찾고', '세', '개가', '되고', '네'],
 ['개가', '되고', '네', '개가', '되고', '어둠은'],
 ['내가', '도무지', '이해가', '안', '되고', '용서가'],
 ['어떤', '기준으로', '어떤', '사랑은', '되고', '어떤'],
 ['되고', '어떤', '건', '안', '되고', '사랑이'],
 ['놀다', '보면', 'Hip', 'up', '되고', '자'],
 ['쫌', '간직해', '멋', '즉', '되고', '싶어'],
 ['맘대로', '안되네', '만약이', '현실이', '되고', '현실이'],
 ['바래다줬어', '하지만', '뭔가', '잘못', '되고', '있었어'],
 ['첫', '고백을', '누군가에게', '하게', '되고', '또'],
 ['보니', '다른', '곳도', '가게', '되고', '항상'],
 ['아니야', '난', '너의', '기쁨이', '되고', '싶을'],
 ['얼어붙어', '움직일', '수가', '없게', '되고', '저'],
 ['아', '한마디', '말이', '노래가', '되고', '시가되고'],
 ['자보고', '살도', '안', '빼도', '되고', 'wonderful'],
 ['부리던', '게', '그리워', '혼자가', '되고', '나서'],
 ['YOU', '너는', '나의', '세상이', '되고', '이'],
 ['너를', '위해', '잘난', '남자', '되고', '싶어'],
 ['사랑해', '오래도록', '서로', '전부가', '되고', '내편이'],
 ['one', '너만의', '사랑', '네가', '되고', '싶다고'],
 ['사랑한다고', '덧없는',

In [41]:
wannabe = before_word(female_t, '싶어')
common(unique(wannabe))

NameError: name 'common' is not defined

In [ ]:
wannabe = before_word(male_t, '갖고')
unique(wannabe)

In [ ]:
common(unique(before_next_word(female_t, '예뻐')))

In [68]:
unique(before_next_word(song_t, '소녀처럼'))

[]

In [ ]:
unique(before_next_word(song_t, '소년처럼'))

In [ ]:
wannabe = before_word(male_t, '되고')
unique(wannabe)

In [ ]:
# search_word 앞에 무슨 단어가 있는지 보는 함수
def before_word(tokenized_list, search_word):
    indices2 = [i for i, x in enumerate(tokenized_list) if x == search_word]
    i = 0
    result = []
    for i in range(len(indices2)):
        next_word_2 = tokenized_list[indices2[i] - 2:indices2[i] + 1]
        result.append(next_word_2)
        i+=1
    return result
    print(result)

In [ ]:
unique(before_next_word(female_t, '공주'))

In [146]:
unique(before_next_word(female_t, '왕자'))

[['멋진', '남자', '고마워', '내겐', '백마탄', '왕자', '이제부터', '우리', '둘이', '너와'],
 ['멋졌나', '하얀', '회전목마', '백마를', '탄', '왕자', '위로', '자꾸', '니가', '겹쳐'],
 ['내', '앞에', '나타난', '백마', '탄', '왕자', '바로', '너', 'It', 's'],
 ['잠이', '들', '때', '백마', '탄', '왕자', '달콤한', '키스로', '잠자는', '공주를'],
 ['버린', '거야', '버린', '거야', '나의', '왕자', '영원히', '내', '눈은', '너만'],
 ['Hus', 'h', '꿈', '속에선', '나만의', '왕자', 'You', 're', 'Mine', 'You'],
 ['친구는', '표범', '치타', '동물의', '왕국', '왕자', '타잔', 'You', 'Say', '아으아으아으아으아'],
 ['친구는', '표범', '치타', '동물의', '왕국', '왕자', '타잔', 'You', 'say', '아으아으아으아으아']]

In [147]:
unique(before_next_word(female_t, '립스틱'))

[['whis', 'key', '술잔에', '남긴', '아슬아슬한', '립스틱', '한', '모금', '더', '깊숙이'],
 ['보이시한', '옷차림', '그랬던', '내가', '하이힐', '립스틱', '그리고', '짧은', '치마', '하나'],
 ['Paint',
  'your',
  'love',
  'Everybody',
  '빨간',
  '립스틱',
  '바르고',
  'Paint',
  'your',
  'love'],
 ['청바지', '원피스', '예쁘다', '할', 'orange', '립스틱', '느낌이', '와', '핑', '돌아봐'],
 ['날', '모르기에', '짧아진', '치마', '빨간', '립스틱', '풀어', '해쳐진', '머리도', '길어진'],
 ['my', 'special', 'holiday', '조금', '짙어진', '립스틱', '좀', '묘하게', 'Mys', 'tic'],
 ['닦아내고', '싶다가도', '네', '옷깃에', '묻은', '립스틱', '생각에', '너를', '절대', '용서'],
 ['돌아올', '수', '있게', '새로', '산', '립스틱', '꺼내서', '전보다', '더', '짙게'],
 ['해롱', '메롱', 'For', 'My', 'Love', '립스틱', '스틱', '세우고', '립스틱', '스틱'],
 ['My', 'Love', '립스틱', '스틱', '세우고', '립스틱', '스틱', '세우고', '어찌나', '눈이'],
 ['보여', '아슬아슬', '다가가서', '훔치고', '싶어', '립스틱', '스틱', '세우고', '내', '입술에'],
 ['하얀', '셔츠', '위에', '살짝', '묻히고', '립스틱', '스틱', '세우고', '네', '입술에'],
 ['널', '이러지마', '오늘', '처음', '봤잖아', '립스틱', '스틱', '세우고', '내', '입술에'],
 ['기억들이', '나를', '찾아와', '클라라의', '꿈', '립스틱', '스틱', '세우고', '립스틱', '스

In [148]:
unique(before_next_word(male_t, '립스틱'))

[['난', 'Mr', 'Mis', 'ery', '빨간', '립스틱', '짧은', '스커트', 'Uh', 'Uh'],
 ['기분좋은', '날만', '즐겨하던', '그입술에', '짙은', '립스틱', '마지막인데', '이렇게', '헤어지는게', '오히려'],
 ['흔한', '주인공처럼', '뻔뻔히', '다가갔어', '빨간', '립스틱', '까만', '스타킹', '장미', '한'],
 ['본', '것도', '아냐', '니', '살구색', '립스틱', '뭔가', '달라', '보여', '오늘'],
 ['그', '존재가', '아름다워', '도도한', '입술', '립스틱', '까칠한', '표정', '얼음', '눈동자'],
 ['입술', '위로', '미끄러지듯', '닿은', '그', '립스틱', '오늘', '밤은', '물끄러미', '나를'],
 ['좋아요', '핸드백', '손수건과', '따뜻한', '코트', '립스틱', '그녀', '구두까지도', '그녀', '곁에'],
 ['touch', 'me', '심장은', '찌릿찌릿해', '빨간', '립스틱', 'I', 'want', 'it', 'baby'],
 ['t', 'want', 'you', 'back', 'uh', '립스틱', '짙게', '바른', '짧은', '치마를'],
 ['치마', '짙어진', '입술', '위', '새빨간', '립스틱', '감출', '수가', '없는', '너의'],
 ['붙는', '원피스', '긴', '생머리에', '새빨간', '립스틱', '때', '타지', '않은', '새하얀']]

In [163]:
unique(before_next_word(female_t, '치마'))

[['내', '다릴', '봐', '예쁘잖아', '짧은', '치마', '입었잖아', '관심있게', '보란말야', 'oh'],
 ['내', '다릴', '봐', '예쁘잖아', '짧은', '치마', '입었잖아', '집에', '가기', '싫어질걸'],
 ['내가', '하이힐', '립스틱', '그리고', '짧은', '치마', '하나', '둘씩', '변해가', '알'],
 ['away', 'Give', 'it', 'away', 'now', '치마', '좀', '내려', '딱', '달라'],
 ['지금껏', '불편해서', '자주', '입지', '않던', '치마', '이제는', '네게', '잘', '보이려'],
 ['show', 'time', '노란', '생머리', '핫핑크색', '치마', '잘', '어울릴', '것', '같지'],
 ['나오려해', '나도', '날', '모르기에', '짧아진', '치마', '빨간', '립스틱', '풀어', '해쳐진'],
 ['전화', '안', '하기만', '해봐', '짧은', '치마', '입고', '친구들과', '술', '마시고']]

In [162]:
unique(before_next_word(male_t, '치마'))

[['good', 'mood', '필요', '없어', '비싼', '치마', '높은', '구두', '필요', '없어'],
 ['띄어', '한눈에', '알아봤어', '검은', '짧은', '치마', 'highheel', '이', '위태로워', '보인걸'],
 ['안했어', '오늘따라', '유난히', '더', '예뻐', '치마', '입었네', '불편하다며', '못보던', '옷인데'],
 ['아가씨', '하이힐', '그', '끝으로', '오르는', '치마', '차마', '말로는', '못할', '아무도'],
 ['여자니까', '안', '그런척하며', '너의', '짧은', '치마', '위를', '모두', '신경', '쓰고'],
 ['내', '눈에', '자꾸', '거슬려', '짧아지는', '치마', '짙어진', '입술', '위', '새빨간'],
 ['너와', '나의', '거리보다', '짧아', '보이는', '치마', '우리보다', '깊어', '보이는', '노출'],
 ['얼마나', '아름답게', '써질지', '실바람', '날리는', '치마', '끝에', '영혼은', '벌써', '널'],
 ['듣는', '내', '속', '뒤틀려', '짧은', '치마', '입지마', '내', '생각이지만', '넌']]

In [165]:
unique(before_next_word(female_t, '하이힐'))

[['표정에', '보이시한', '옷차림', '그랬던', '내가', '하이힐', '립스틱', '그리고', '짧은', '치마'],
 ['쳐다', '본다고', '글리터', '잔뜩', '뿌린', '하이힐', '신고서', '사탕을', '입에', '물고'],
 ['girl', 'Glam', 'glam', 'girl', '높은', '하이힐', '꽤', '진한', '아이라인', '머리부터'],
 ['만들거야', '사랑해', '너하나만', '굽', '높은', '하이힐', '날', '좀', '봐요', '그대가'],
 ['그대가', '좋아한다', '해서', '신었잖아요', '섹시한', '하이힐', '빨주노초파남보', '아무거나', '골라봐', '널'],
 ['해쳐진', '머리도', '길어진', '손톱', '길어진', '하이힐', '매일', '취기에', '살아', '어지러워진'],
 ['반짝거리는', '불빛의', '유혹', '아찔한', '내', '하이힐', '들썩거리는', 'Party', 'Mama', 'She'],
 ['Oh', 'Wait', 'up', 'girl', '삐딱한', '하이힐', '걸음걸인', '삐뚤빼뚤', 'Grow', 'up'],
 ['하고', '다리가', '아파도', '굽', '높은', '하이힐', '신고', '너에게', '잘', '보이려고'],
 ['뭐', '그건', '내가', '신은', '높은', '하이힐', '잘', '봐', '완벽해', '보이는'],
 ['완벽해', '보이는', '비밀', '날', '봐', '하이힐', '하나로', '모든', '게', '달라'],
 ['잘빠져', '보이는', '비밀', '날', '봐', '하이힐', '하나로', '모든', '게', '달라'],
 ['노출', '안', '해도', '아찔한', '나의', '하이힐', '새까만', '스타킹', '도저히', '눈을'],
 ['옷을', '빼입고', '오늘따라', '조금', '높은', '하이힐', '어색하지만', '들떠버린', '걸음걸이', '점점'],
 ['널', '만나러', '가',

In [164]:
unique(before_next_word(male_t, '하이힐'))

[['she', '비친', '실루엣', '도도한', '날카로운', '하이힐', '소리', '뜨거워진', '심장을', '찌르고'],
 ['니', '겉모습에는', '말야', '검정스타킹에', '아찔한', '하이힐', '그래', '그것은', '사기야', '근데'],
 ['좋아', '다만', '조금', '신경', '쓰인', '하이힐', '아', '모르겠어', '너의', '진짜'],
 ['So', 'lis', 'ten', 'baby', '하얀', '하이힐', '살짝', '가린', '레이스', '스커트'],
 ['보내주긴', '싫어', '걸', '벗어둬', '니', '하이힐', '오늘', '가기', '전에', 'I'],
 ['보내주긴', '싫어', '걸', '벗어둬', '니', '하이힐', '오늘', '가기', '전에', 'yeah'],
 ['갖고', '싶은', '걸', '벗어둬', '니', '하이힐', '오늘', '가기', '전에', 'baby'],
 ['싫어', '보내주긴', '싫어', '벗어둬', '니', '하이힐', '오늘', '가기', '전에', 'baby'],
 ['그냥', '즐겨', '걸', '벗어둬', '니', '하이힐', '오늘', '가기', '전에', '그리워'],
 ['그어왔던', '친한', '친구라는', '선', '니', '하이힐', '뒤에', '넌', '내', '바로'],
 ['맨', '얼굴이', '좋아', '화장하지', '마', '하이힐', '신지', '않아도', '돼', '그대로가'],
 ['don', 'belong', 'to', 'you', '아가씨', '하이힐', '그', '끝으로', '오르는', '치마']]

In [43]:
unique(before_next_word(song_t, '셔츠'))

[['내게', '맡겨', '땀에', '젖은', '트레이닝', '셔츠', 'G', 'L', 'A', 'M'],
 ['짙은', '향이', '아닌', '비누같은', '은은함이', '셔츠', '사이로', '나의', '맘속에', '살며시'],
 ['취향이니까', '예민하지마', '컴플렉스', 'Nothing', '슬렉스', '셔츠', '로퍼', '클래식한', '스타일로', '난'],
 ['간절한', 'feelin', '흰', '운동화', '이', '셔츠', '전부', '젖어도', '돼', '머리'],
 ['s', 'dance', '벗어버려', 'No', 'heels', '셔츠', '위엔', 'No', 'tie', '숨'],
 ['s', 'go', 'Date', '할', '때', '셔츠', '입는', '건', '어때', '불러줄게'],
 ['입술에', '바르고', '잘', '다려진', '하얀', '셔츠', '위에', '살짝', '묻히고', '립스틱'],
 ['넥타이', '심쿵', '내려앉은', '기분', '흰', '셔츠', '위에', '묻혀', '보고', '싶은'],
 ['너의', '시선강탈', '난', '너의', '심장폭행', '셔츠', '사이로', '보일', '듯', '말'],
 ['남자들보다', '괜찮고', '어제', '새로', '산', '셔츠', '색상도', '푸른', '하늘과', '완벽한'],
 ['않을', '거야', '잠깐만', '내가', '사준', '셔츠', '검은', '넥타이', '놀랐어', '네가'],
 ['되버렸나봐요', '아무것도', '할', '수가', '없네요', '셔츠', '하나', '조차도', '어디', '있는지']]

In [52]:
unique(before_next_word(female_t, '다리'))

[['눈', '코', '입', '표정도', '팔', '다리', '걸음도', '내', '말을', '듣지'],
 ['못난이', 'what', '자꾸', '봐', '내', '다리', '진짜', '너', '내', '타입'],
 ['안', '돼', '내', '머릿속을', '뛰어다니느라', '다리', '아플', '텐데', '내', '옆에'],
 ['내', '뒤로', '줄서', 'MY', 'BODY', '다리', '쉑', '라리다리', '짜증이', '나겠지'],
 ['내말좀', '들어봐', '이리와', '딱좋아', '죽이는데', '다리', '섹시한데', '거기', '너희들의', '이름'],
 ['나', '매력', '있는', '눈', '날씬한', '다리', '날', '바라보잖아', 'Feeling', 'me'],
 ['나나', '나나', '나나', 'No', '팔', '다리', '다리', '머리', '허리', '허리'],
 ['나나', '나나', 'No', '팔', '다리', '다리', '머리', '허리', '허리', '허리'],
 ['it', '내', '머리', '어깨', '무릎', '다리', '완벽하게', '또', '악세사리', 'make'],
 ['멀리', '멀리가', '내', '머리', '어깨', '다리', '다리가', '짙은', '안개', '속을'],
 ['턱선', '벨', '듯한', '콧날', '허리', '다리', '내', '치명적인', '힙', '라인'],
 ['다른', '사람', '있는지', 'Hey', 'you', '다리', '예쁜', '여자가', '좋아', '코가'],
 ['잠들', '때까지', '쉬지않는', '내', '팔과', '다리', '머리', '허리', '내', '몸이'],
 ['돼', '예쁘장한', '나', '귀여워', '통통한', '다리', '나', '애기', '같대', '나는'],
 ['몰라', '깜찍한', '앵두빛', '입술부터', '힙', '다리', '한껏', '뽐을', '내면', '모든'],
 ['out', 'of', 'here', '나'

In [57]:
import re

In [62]:
unique(before_next_word(male_t, '허리'))

[['못', '참겠어', 'Oh', '넌', '머리부터', '허리', 'to', 'the', '다리까지', '다'],
 ['갸냘픈', '너의', '어깨', '잘록한', '너의', '허리', '빵빵한', '너의', '엉덩이', '뭘'],
 ['어딜', '가든', '내', '핸드백은', '니', '허리', 'Yo', 'ma', 'honey', '볼'],
 ['미형', '문화재', '그녀', '머리', '바디', '허리', '다리', '말', '못하는', '범위까지'],
 ['지웠어', 'Sorry', 'Sorry', '쭉빠진', '다리', '허리', '니', '눈빛에', 'hey', '격하게'],
 ['Let', 'it', 'go', '쭉빠진', '다리', '허리', '니', '눈빛에', 'hey', '격하게'],
 ['now', '넌', '감동이야', '머리', '어깨', '허리', '다리', '넌', '감동이야', '살며시'],
 ['시', '넌', '감동이야', '머리', '어깨', '허리', '다리', '넌', '감동이야', '살며시'],
 ['em', '넌', '감동이야', '머리', '어깨', '허리', '다리', '넌', '감동이야', '살며시'],
 ['Down', '눈을', '못', '떼겠어', '너의', '허리', '라인에', '종일', '소용돌이', '쳐'],
 ['긴', '생머리', '얇은', '다리', '가는', '허리', '어우러진', '너의', '목소리', '내'],
 ['말았어', '허리까지', '내려오는', '머리', '잘록한', '허리', '그', '허리', '아래보면', '늘씬한'],
 ['내려오는', '머리', '잘록한', '허리', '그', '허리', '아래보면', '늘씬한', '종아리', '열심히'],
 ['계속해', '더', '계속', '흔들리는', '니', '허리', 'Ma', 'honey', '푸대접', '하지는'],
 ['않는', '딴따라', '랄라라', '따라', '하시다가', '허리', '다', '나가', '정통'

In [151]:
unique(before_next_word(female_t, '화장'))

[['처음', '해', '본', '이', '눈', '화장', '나', '오늘', '예뻐', '맞출게'],
 ['변한듯해', '눈치', '챈지', '오래야', '머리모양', '화장', '바꿔봐도', '별', '관심', '없잖아'],
 ['원치', '않았기에', '참', '다행이죠', '난', '화장', '안', '한', '얼굴로', '그대'],
 ['이제는', '못', '알아볼', '정도까지', '진해버린', '화장', '낯설게만', '느껴지던', '남자들의', '시선'],
 ['영원할', '꺼야', '짧게', '자른', '머리', '화장', '안한', '맨', '얼굴', '좀'],
 ['I', 'm', 'a', 'lazy', 'girl', '화장', '따윈', '하지', '않죠', '뻗친'],
 ['그', '때로', '돌아가고', '싶어', '가고파', '화장', '안', '하고도', '깨끗했었던', '철부지'],
 ['야', '야', '야', '잠깐', '나', '화장', '좀', '고칠게', '좋은', '기억을'],
 ['우', '다른', '건', '다', '모르면서', '화장', '안한건', '또', '아네요', '내가'],
 ['우', '다른', '건', '다', '모르면서', '화장', '안한건', '또', '아네요', '늑대들은']]

In [149]:
unique(before_next_word(male_t, '화장'))

[['밝아요', '너는', '뭔가', '너무', '맑아요', '화장', '하나', '안해도', '눈에', '띄잖아'],
 ['살이', '쪘다며', '이', '바보야', '너', '화장', '지워도', '예뻐', '뭐를', '입어도'],
 ['감추고', '있단', '걸', '알고', '있나요', '화장', '지워도', '예뻐', '뭐를', '입어도'],
 ['자꾸만', '좋아져', '향수를', '뿌리지', '않아도', '화장', '고치지', '않아도', '난', '항상'],
 ['널', '사랑해', '향수를', '뿌리지', '않아도', '화장', '고치지', '않아도', '난', '항상'],
 ['거울에', '보이는', '나는', '며칠', '밤낮의', '화장', '덕에', '상당히', '상한', '피부와'],
 ['웃는', '얼굴', '보고', '싶어', '왜', '화장', '번져', '버린', '거야', '쇼윈도에'],
 ['내겐', '완벽한', '사람인걸', '숨만', '쉬어도', '화장', '안', '해도', '넌', '아름다워'],
 ['항상', '예쁘다고', '내가', '말했었잖아', '넌', '화장', '안해도', '돼', '필요', '없어'],
 ['널', '위해서라면', 'Oh', 'my', 'girl', '화장', '안', '한', '쌩얼', '뿔테'],
 ['굽을', '벗어', '던지고', '운동화', '신고', '화장', '안', '해도', '그래도', '너무'],
 ['피부까지도', '내', 'Style', '청순해', '보이는', '화장', '도도한', '표정은', '내', '심장을']]

In [170]:
unique(before_next_word(female_t, '옷'))

[['내', '일이야', '매일', '다른', '예쁜', '옷', '입고서', '화려한', '조명에', '신이'],
 ['oh', 'my', 'girl', '니가', '예쁘다던', '옷', '입고', '네', '앞에', '있는'],
 ['불안하게', '생각', '말아요', '가끔은', '옛날', '옷', '입고', '싶은', '것처럼', '아니'],
 ['피치', '만인의', '파이는', 'Wifi', '소시는', '옷', '가게에서', '티파니', '소가', '위로'],
 ['좋아', '보여', '오늘따라', '왠지', '입을', '옷', '고르기가', '힘드네', '내게', '할'],
 ['위선이', '난', '너무나', '웃겨', '이런', '옷', '이런', '머리', '모양으로', '이런'],
 ['반지', '아니야', '내', '차', '내', '옷', '내가', '벌어서', '산', '거야'],
 ['좀', '내려', '딱', '달라', '붙는', '옷', '좀', '버려', '니', '기분'],
 ['두', '시간이지나', '옷장에', '옷들도', '이', '옷', '저', '옷', '모두', '꺼내서'],
 ['옷장에', '옷들도', '이', '옷', '저', '옷', '모두', '꺼내서', '입어보며', '한껏'],
 ['머릴', '그녀처럼', '좀', '바꿀까요', '그녀처럼', '옷', '입을까요', '이런', '내맘', '참'],
 ['문을', '박', '차고', '발칙한', '섹시한', '옷', '갈아', '입고', '나를', '원한다면'],
 ['살', '수가', '없어', '일년에', '입을', '옷', '며칠', '다', '입고', '너를'],
 ['수가', '없어', '없어', '일년에', '입을', '옷', '몇', '일만에', '입고', '너를'],
 ['예쁜', '별명', '짓고', '장난치고', '멋진', '옷', '서로', '가끔', '바꿔', '입고'],
 ['주는', '넌', '쇼윈도', '너머', '멋진', '옷'

In [171]:
unique(before_next_word(male_t, '옷'))

[['괌', '그리고', '바하마', '비싼', '시계', '옷', '이젠', '다', '귀찮아', '이젠'],
 ['발걸음은', '베이스', '리듬에', '모두', '맞춰', '옷', '매무새를', '차례로', '갖춰', '비틀즈처럼'],
 ['너무', '좋네', '사람들이', '반해', '너의', '옷', '스타일에', '너의', '헤어스타일에', '마네킹'],
 ['수십병이', '넘는걸', '너와', '만날때', '입던', '옷', '모두', '다', '버렸어', '너와'],
 ['많이', '외쳐주셔', '가지구', '저희들이', '바로', '옷', '갈아', '입고', '스타일리스트', '누나들한테'],
 ['아침엔', '너란', '향길', '씻어', '새', '옷', '입고', '준비를', '해', '홀로'],
 ['있던', '그', '곳', '입던', '그', '옷', '내게', '줬던', '너에', '미소가'],
 ['오늘은', '코끼리', '코하고', '마셔도', '돼', '옷', '차림새는', '신경', '쓰지마', '쟤는'],
 ['평생', '만들게', '해', '김남준', '컬렉션', '옷', '색깔은', '까만색으로', '이제야', '좀'],
 ['주식이라면', '투자를', '하지', '너와', '입을', '옷', 'choice', '옷장을', '포위', '내'],
 ['우리', '둘이', '둘이', '맞춰', '입던', '옷', '모든', '게', '다', '그대로'],
 ['붉히면서까지', '만날', '일', '없어', '더러워진', '옷', '털어봤자', '먼지만', '나', '상처가'],
 ['같아', 'Feel', 'good', 'feel', 'good', '옷', '색깔마저', '내', '맘에', '들어'],
 ['해야', '돼', '인터넷에다', '물어볼까', '어떤', '옷', '또', '어떤', '곳', '공부를'],
 ['그때', '우리가', '내가', '기억난게', '빨간', '옷', '입고', 'Firs', 't', 

In [175]:
unique(before_next_word(female_t, '바지'))

[['천천히', 'Slowly', '그러다', '체하지', '니', '바지', '먹은', '특정', '부위', '스키니바지'],
 ['마요', '잘못돼요', '너', '꽉', '낀', '바지', '긴', '생머리', '했다고', '좀']]

In [177]:
unique(before_next_word(male_t, '바지'))

[['반지는', '언제든', '낄', '수', '있게', '바지', '주머니', '안에', '넌', '왜'],
 ['봤다면', '버섯', '먹고', '크게', '자라', '바지', '위에', '팬티를', '입고', '망토를'],
 ['가고파', '난', 'Yeah', 'uh', '촌스러운', '바지', '엄마가', '입혀주는', '대로', '입던'],
 ['몇몇은', '아마', '오줌을', '지렸어', '너', '바지', '짚업이', '안되', '삐져서', '나온'],
 ['서울', '올라왔지', '난', '똥', '싼', '바지', '정상을', '원하지만', '현실은', '바닥'],
 ['death', 'of', 'art', '살짝', '내린', '바지', 'snapback', '이', '필수', '목에']]

In [179]:
unique(before_next_word(female_t, '시계'))

[['마', '상상', '속에', '빙빙', '돌아가는', '시계', '속에', '우리', '사인', '빛이'],
 ['듯', '해', '서글픈', '눈물뿐이네', '멈춰진', '시계', '바늘처럼', '모든', '게', '멈춘'],
 ['왜', '지각을', '해요', '돌아가는', '작은', '시계', '위에', '발을', '맞춰', '바삐'],
 ['그대가', '날', '기다려줘요', '돌아가는', '작은', '시계', '위에', '발을', '맞춰', '바삐'],
 ['되겠어', '아', '아무래도', '안', '되겠어', '시계', '초침', '가는', '소리까지', '나를'],
 ['보면', '죽어', '나를', '만날', '때는', '시계', '보면', '죽어', '내', '앞에서'],
 ['입고', '조용히', '창문을', '열면', '의미없는', '시계', '사라져', '잊혀진', 'radio', '정적에'],
 ['빨간', '불이', '켜져도', '멈추지', '마', '시계', '소리에', '움직임을', '맞춰', '서툰'],
 ['분', '초', '모두', '나누고', '싶은', '시계', '바늘', '향하는', '그', '모든'],
 ['없는', '거짓말에', '뻔한', '그', '레퍼토리', '시계', '소리', 'Tic', 'toc', 'tic'],
 ['두', '눈은', '아직도', '감기고', '잠깐', '시계', '바늘', '멈출', '순', '없을까'],
 ['다들', '바쁜가', '봐', '주위에', '가득한', '시계', '소리', '아무도', '없고', '휘파람이'],
 ['이', '소리가', '들리니', '내', '안의', '시계', '초침', '잔인할', '만큼이나', '칼'],
 ['안겼던', '순간', '쉼', '없이', '돌아가는', '시계', '바늘', '위로', '그대와의', '추억들이'],
 ['낙서만', '수리', '수리', '수리', '마수리야', '시계', '바늘아', '느리게', '구르르', '빙글'],


In [178]:
unique(before_next_word(male_t, '시계'))

[['타고', '괌', '그리고', '바하마', '비싼', '시계', '옷', '이젠', '다', '귀찮아'],
 ['너란', '존재가', '내게', '후회', '됐어', '시계', '태엽처럼', '다시', '돌리고', '싶어'],
 ['시간은', 'tic', 'toc', '흐르고', '있고', '시계', '속', '분침은', '니가', '있는'],
 ['더', '먼저', '나를', '깨우던', '그', '시계', '소리도', '오늘만은', '울지', '않을거야'],
 ['때', '그대와', '보냈던', '시간들보다', '작은', '시계', '바늘은', '변함없이', '한', '바퀴를'],
 ['Bounce', 'Bounce', 'Make', 'It', 'Bounce', '시계', '따윈', '보지', '말고', 'Day'],
 ['머린', '차가워져', '가슴은', '더욱', '뜨거워져', '시계', '태엽', '돌아가듯', '끝은', '정해져'],
 ['잊을', '수', '없어', '다시', '돌아와', '시계', '소리에', '눈을', '뜨면', '새롭지만'],
 ['들어가자', '잠깐', '예민한', '날이', '선', '시계', '소리', '나는', '방', '궁금증에'],
 ['맘', '너는', '알까', 'Oh', 'No', '시계', '바늘을', '멈춰버릴', '순', '없을까'],
 ['무지갠', '지난', '힘든', '일들', '버려줄', '시계', 'say', 'two', 'in', 'the'],
 ['party', 'Let', 's', 'party', '보지마', '시계', '이', '뜨거운', '땀', '속의'],
 ['Check', 'it', 'One', 'Two', 'Three', '시계', '바늘만', '쳐다', '보는게', '말'],
 ['들어서면', '번', '울리는', '내', '뻐꾸기', '시계', '그', '시간들이', '거짓없이', '자랑스럽게'],
 ['소중한', '기회', '끝도', '없이', '돌아가는', '시계',

In [181]:
unique(before_next_word(female_t, '머리'))

[['Knock',
  'knock',
  '들어간다',
  'Soyeon',
  '이',
  '머리',
  '위에',
  'Spotlight',
  '저기',
  '잘난척하던'],
 ['there', '우리밖에는', '없어', '지금', '내', '머리', '위론', '조금만', '더', '위로'],
 ['그대', '같은', '사람', '없어요', '오늘도', '머리', '화장을', '하고', '친구들과', '만나'],
 ['매일', '그대', '숨결과', '목소리는', '내', '머리', '속에', '그려져', 'Um', '하나에서'],
 ['알고', '싶어', 'baby', '날', '따라다니는', '머리', '속에', '물음표', '거긴', '대체'],
 ['듯', '열이', '종일', '돌고', '내', '머리', '위론', '너만', '핑글', '핑글'],
 ['시간', '가지', '참을성', '없는', '나야', '머리', '쓰지', '용을', '써도', '답은'],
 ['집', '앞이래', 'Eh', '질끈', '묶어낸', '머리', '한', '듯', '안', '한'],
 ['Sweet', '몰래', '뿌린', '향수가', '아', '머리', '아파', 'I', 'm', 'Not'],
 ['하늘을', '바라보며', '흥얼거리며', '콧노래를', '불러', '머리', '속에', '아름다운', '너를', '그려'],
 ['하나', '둘씩', '복잡한', '눈치', '게임', '머리', '아픈', '나니까', '서둘러', '재미없어'],
 ['헤매이잖아', '밤이며', '낮이며', '추억은', '상관없이', '머리', '위를', '온통', '다', '뒤덮어'],
 ['봐', '밤이며', '낮이며', '추억은', '상관없이', '머리', '위를', '온통', '다', '뒤덮어'],
 ['아니야', '사랑은', '아야야야', '야야', '아야야야', '머리', '어깨', '무릎', '발', '무릎'],
 ['발', '무릎이', '아야야야', '야야', '

In [180]:
unique(before_next_word(male_t, '머리'))

[['그녀를', '관찰하지', '아름답게', '빗어', '내린', '머리', '거기서', '나는', '음', '상쾌한'],
 ['한', '사람만을', '영원토록', '미치도록', '검은', '머리', '파뿌리', '되도록', '사랑한다는게', '도무지'],
 ['돈', '밖에', '없어', '돈', '돈', '머리', '까지', '돈', '불쌍한', '사람들이'],
 ['그', '예쁜', '입술을', '훔칠거야', '우리', '머리', '위에', '빛나는', '별들', '너무나'],
 ['해도', '멍하니', '난', '걷고', '있어', '머리', '속에', '넌', '매일', '출연해'],
 ['될', '거야', '다들', '두', '손을', '머리', '위로', '들어봐', 'like', 'this'],
 ['know', 'that', 'I', 'got', 'em', '머리', '아픈', '일', '더', '생각마'],
 ['the', 'world', '아무도', '감당', '못해', '머리', '위에', 'Sun', '올라와도', 'Turn'],
 ['올라가', 'We', 're', 'going', 'up', '머리', '아픈', '일', '더', '생각마'],
 ['G', '이리저리', '생각', '많아', '내', '머리', '무거워', '무거워', '너로', '가득'],
 ['행복이', '우선되야', '한다', '그래서', '검은', '머리', '파뿌리', '되도록', '너만을', '사랑해주고싶다'],
 ['얘기', '같았죠', '그러던', '어느날', '내', '머리', '위에도', '축복이', '내린거죠', '온'],
 ['믿지', '못했죠', '그러던', '어느날', '내', '머리', '위에도', '축복이', '내린거죠', '온'],
 ['m', 'road', 'Runner', 'fas', 't', '머리', '속은', '까맣게', '널', '잊어버리고'],
 ['하루를', '지나', '밤새', '니', '생각에', '머리', '속에', '

In [167]:
unique(before_next_word(female_t, '교복'))

[['너를', '기억하고', '있어', '나', '근사하게', '교복', '잘', '어울리던', '너', '어딘지'],
 ['삶이', '아닌데', '그', '때마다', '생각나죠', '교복', '입은', '그', '때', '뭘']]

In [166]:
unique(before_next_word(male_t, '교복'))

[['정말', '행복해질까', '놀고', '먹고', '싶어', '교복', '찢고', '싶어', 'Make', 'money'],
 ['본', '순간', '단발에', '너무', '예쁜', '교복', '그', '모습만', '생각나', '아무데도'],
 ['운동화는', 'oh', '아찔한', '하이힐로', '단발머리', '교복', '치만', '생머리의', '미니스커트', 'Say'],
 ['보고싶어', '등교길에', '조잘대며', '걷는', '여자애들', '교복', '단정한', '긴머리', '소녀', '낯설지가'],
 ['생각해', '함께인', '너와나', '언제나', '깔끔한', '교복', '또', '남자들', '틈에서', '너무']]

In [80]:
unique(next_word(male_t, '야'))

[['야', '어이없이넌', '얘기했', '지', '정신이'],
 ['야', '넌', '정말안돼겠다', '널이해', '못하겠다'],
 ['야', '큰', '용기일', '것', '같다'],
 ['야', '조용', '토니', '계속해', '장'],
 ['야', '근데', '지금', '몇시야', '분'],
 ['야', '나가야', '나가야', '되는거', '아냐'],
 ['야', '눈썹봐라', '와', '속눈썹', '봐'],
 ['야', '근데', '너', '이름', '뭐야'],
 ['야', '이름', '참', '거창하다', '우주야'],
 ['야', '멋지다', '하나는', '경찰이요', '경찰'],
 ['야', '너희들', '우리집에', '왜', '온'],
 ['야', '너희들', '우리집에', '왜', '온거야'],
 ['야', '고정관념이', '문제야', '이제부터', '깨'],
 ['야', '친구들이', '그러는데', '스키장에', '가면'],
 ['야', '헝그리하다', '앞만', '보고', '달려온'],
 ['야', '저', '세상을', '향해', '소리질러'],
 ['야', '해', '네', '모든', '걸'],
 ['야', '해', '너의', '사랑', '지워'],
 ['야', '야', '차가워진', '너를', '마지막'],
 ['야', '차가워진', '너를', '마지막', '모습으로'],
 ['야', '해', '네', '모든걸', '잡아야만'],
 ['야', '누가', '뭐래도', '우리', '추억은'],
 ['야', '너만을', '사랑해', '난', '알아'],
 ['야', '이럴수록', '내', '가슴만', '더'],
 ['야', '똑똑한', '쟨', '전교', '등해'],
 ['야', '뜨거운', '이밤', 'Don', 't'],
 ['야', '너', '빨리', '일어나', '야'],
 ['야', '너', '더', '질러봐', '야'],
 ['야', '너', '빨리', '일어나', '힘껏'],
 ['야', '우리가', '년동안', '활동하면서

In [48]:
iwilldo_f = before_word(female_t, '해줄게')
iwilldo_f

NameError: name 'before_word' is not defined

In [51]:
unique(before_word(female_t, '해줄게'))  # 여자 아이돌 가사 중 '해줄게' 앞 맥락 보기

[['다시', '믿게', '해줄게'],
 ['곁에만', '있게', '해줄게'],
 ['쿵', '하게', '해줄게'],
 ['이라면', '떠나게', '해줄게'],
 ['너를', '웃게', '해줄게'],
 ['만들어줄게', '그렇게', '해줄게'],
 ['항상', '웃게', '해줄게'],
 ['Alright', '알게', '해줄게'],
 ['대로', '오늘', '해줄게'],
 ['특별히', '뭐든', '해줄게'],
 ['특별히', '허락', '해줄게'],
 ['다시', '느끼게', '해줄게'],
 ['언제까지', '함께', '해줄게'],
 ['오늘처럼만', '웃게', '해줄게'],
 ['후회하지', '않게', '해줄게'],
 ['너를', '깨어나게', '해줄게'],
 ['음악을', '마시게', '해줄게'],
 ['모두', '잊게', '해줄게'],
 ['모르는', '척', '해줄게'],
 ['않아', '부르게', '해줄게'],
 ['다', '잊게', '해줄게']]

In [52]:
unique(before_word(male_t, '해줄게'))  # 남자 아이돌 가사 중 '해줄게' 앞 맥락 보기

[['그대가', '원하는대로', '해줄게'],
 ['언제나', '행복하게', '해줄게'],
 ['내가', '다', '해줄게'],
 ['너를', '웃게', '해줄게'],
 ['준비했던', '말을', '해줄게'],
 ['난', '뭐든', '해줄게'],
 ['매일매일', '웃게', '해줄게'],
 ['끝까지', '함께', '해줄게'],
 ['U', '잘', '해줄게'],
 ['you', '다', '해줄게'],
 ['마', '내가', '해줄게'],
 ['듣고', '느끼게', '해줄게'],
 ['뭐든', '다', '해줄게'],
 ['때로', '간다면', '해줄게'],
 ['꽃길만', '걷게', '해줄게'],
 ['you', '행복하게', '해줄게'],
 ['해줄게', '행복하게', '해줄게'],
 ['해줘', '행복하게', '해줄게'],
 ['너에게', '다', '해줄게'],
 ['위해', '하나도', '해줄게'],
 ['다', '맞출게', '해줄게'],
 ['안', '하면', '해줄게'],
 ['더', '웃게', '해줄게'],
 ['원하면', '다', '해줄게'],
 ['내가', '웃게', '해줄게'],
 ['뭐든지', '다', '해줄게'],
 ['눈감아줄게', '뭐든지', '해줄게'],
 ['어디라도', '함께', '해줄게'],
 ['제대로', '느끼게', '해줄게'],
 ['매일', '느끼게', '해줄게'],
 ['것보다', '더', '해줄게'],
 ['널', '기쁘게', '해줄게'],
 ['마음을', '설레게', '해줄게'],
 ['말자', '행복하게', '해줄게'],
 ['못', '떼게', '해줄게'],
 ['원한다면', '뭐든지', '해줄게'],
 ['널', '웃게', '해줄게'],
 ['보내기엔', '보내기에는', '해줄게'],
 ['아픔', '아물게', '해줄게'],
 ['생각마', '행복하게', '해줄게'],
 ['않게', '잘', '해줄게'],
 ['듯', '매일', '해줄게'],
 ['bye', '뭐든', '해줄게'],
 ['되게', '라도', '해

In [53]:
unique(before_word(female_t, '돼줄게'))

[['막아주고', '빛이', '돼줄게'],
 ['너의', '여자가', '돼줄게'],
 ['밝은', '빛이', '돼줄게'],
 ['네', '편이', '돼줄게'],
 ['너만의', '천사가', '돼줄게'],
 ['걸', '빛이', '돼줄게'],
 ['골목에', '등불이', '돼줄게'],
 ['바다에', '등대가', '돼줄게'],
 ['앙큼한', '고양이', '돼줄게'],
 ['너만의', '빛이', '돼줄게'],
 ['될게', '그늘이', '돼줄게'],
 ['반짝이는', '별이', '돼줄게']]

In [54]:
unique(before_word(male_t, '돼줄게'))

[['사랑의', '포로가', '돼줄게'],
 ['말해', '다', '돼줄게'],
 ['너만의', '수퍼맨이', '돼줄게'],
 ['bae', '북이', '돼줄게'],
 ['이제는', '위로가', '돼줄게'],
 ['style', '내가', '돼줄게'],
 ['그런', '사람', '돼줄게'],
 ['하루', '쉼표가', '돼줄게'],
 ['아파해줄게', '날개가', '돼줄게'],
 ['꽃', '길이', '돼줄게'],
 ['해줄게', '힘이', '돼줄게'],
 ['내가', '힘이', '돼줄게'],
 ['좋은', '사람이', '돼줄게'],
 ['마지막', '빈칸이', '돼줄게'],
 ['되어', '빛이', '돼줄게'],
 ['너의', '침대가', '돼줄게'],
 ['너의', '불씨가', '돼줄게'],
 ['내가', '빛이', '돼줄게'],
 ['너의', '기적이', '돼줄게'],
 ['너의', '편이', '돼줄게'],
 ['아프다면', '그늘이', '돼줄게'],
 ['괜찮은', '너의남자', '돼줄게'],
 ['위해', '집이', '돼줄게'],
 ['너만의', '별이', '돼줄게'],
 ['포근한', '침대가', '돼줄게'],
 ['밤에', '달이', '돼줄게'],
 ['누구보다', '힘이', '돼줄게'],
 ['탑', '스타', '돼줄게']]

In [55]:
unique(before_word(female_t, '될게'))

[['웃는', '거울이', '될게'],
 ['있는', '사람이', '될게'],
 ['속에', '파도가', '될게'],
 ['그대', '힘이', '될게'],
 ['나', '빛이', '될게'],
 ['난', '꽃이', '될게'],
 ['너의', '램프가', '될게'],
 ['내리면', '우산이', '될게'],
 ['될게', '햇살이', '될게']]

In [27]:
len(unique(b_n_word(song_t, '멋진')))

NameError: name 'unique' is not defined

In [56]:
unique(before_word(male_t, '될게'))

[['위한', '늑대가', '될게'],
 ['너의', '빛이', '될게'],
 ['위한', '주인공이', '될게'],
 ['그대', '방패가', '될게'],
 ['너의', '이상형', '될게'],
 ['baby', '가', '될게'],
 ['힘들면', '깃털이', '될게'],
 ['그대만의', '꽃길이', '될게'],
 ['뒤로', '내가', '될게'],
 ['위한', '별이', '될게'],
 ['너의', '힘이', '될게'],
 ['맞는', '남자가', '될게'],
 ['멋진', '애인이', '될게'],
 ['내가', '우산이', '될게'],
 ['환한', '빛이', '될게'],
 ['집착하는', '남자', '될게'],
 ['위한', '내가', '될게'],
 ['내가', '빛이', '될게'],
 ['위한', '남자가', '될게'],
 ['밝히는', '빛이', '될게'],
 ['그런', '놈이', '될게'],
 ['내가', '별이', '될게'],
 ['않는', '믿음이', '될게'],
 ['하나뿐인', '믿음이', '될게'],
 ['영원한', '친구가', '될게'],
 ['그런', '남자가', '될게'],
 ['싶은', '나무가', '될게'],
 ['작은', '별이', '될게'],
 ['너의', '하늘이', '될게'],
 ['줄', '한사람이', '될게'],
 ['달려갈', '한사람이', '될게'],
 ['네', '편이', '될게'],
 ['좋은', '남자가', '될게'],
 ['대세는', '내가', '될게'],
 ['baby', '화살이', '될게'],
 ['걸', '배워서는', '될게']]

In [48]:
unique(before_word(female_t, '줘봐'))

[['들이고', '확신을', '줘봐'],
 ['나에게', '미소를', '줘봐'],
 ['너만의', '느낌을', '줘봐'],
 ['사랑을', '보여', '줘봐'],
 ['좀', '해', '줘봐'],
 ['영수증', '좀', '줘봐']]

In [66]:
unique(before_word(male_t, '능력이'))

[['나의', 'Olivia', 'Huss', 'ey', '능력이', '있어도'],
 ['뒤', '하나같이', '말해', '넌', '능력이', '좋아'],
 ['봐서라도', '예전처럼', '돌아와주라', '내', '능력이', '부족하다면'],
 ['못생긴', '여자라곤', '하나도', '없어', '능력이', '많은']]

In [ ]:
unique(before_word(female_t, '꿈'))

In [42]:
unique(before_word(male_t, '꿈'))

NameError: name 'unique' is not defined

In [218]:
unique(before_word(female_t, '소녀'))

[['꿈을', '갖고있는', '소녀'],
 ['부탁할게', '어린', '소녀'],
 ['정말', '사랑스런', '소녀'],
 ['Right', '때로는', '소녀'],
 ['너만의', '영원한', '소녀'],
 ['why', '소년', '소녀'],
 ['같아', '소년', '소녀'],
 ['난', '순진한', '소녀'],
 ['마음', '열여덟', '소녀']]

In [219]:
unique(before_word(female_t, '소년'))

[['말했던', '그', '소년'], ['me', 'why', '소년'], ['꿈만', '같아', '소년']]

In [220]:
unique(before_word(male_t, '소녀'))

[['영원히', '사랑해', '소녀'],
 ['단정한', '긴머리', '소녀'],
 ['않게', '성냥팔이', '소녀'],
 ['떠난', '그', '소녀'],
 ['정말', '귀여웠던', '소녀'],
 ['붉게', '수줍었던', '소녀'],
 ['수줍은', '한', '소녀'],
 ['up', '세기', '소녀'],
 ['테지만', '작은', '소녀'],
 ['뱅뱅', '작은', '소녀'],
 ['보면', '수줍은', '소녀']]

In [221]:
unique(before_word(male_t, '소년'))

[['열아홉살', '코흘리게', '소년'],
 ['그래', '빛의', '소년'],
 ['난', '철부지', '소년'],
 ['꿈을', '꾼', '소년'],
 ['옆에서', '놀렸었던', '소년'],
 ['웃고', '있던', '소년'],
 ['말아', '양치기', '소년'],
 ['현실이', '두려운', '소년'],
 ['짓던', '그런', '소년']]

In [222]:
unique(next_word(female_t, '소녀'))

[['소녀', '일', '뿐이야', '너', '정신차려'],
 ['소녀', '꿈을', '짓밟진', '말아', '줘'],
 ['소녀', '당신이', '몰라줄', '뿐', '그대가'],
 ['소녀', '같게', '하나하나', '조심스럽게', '행동하기'],
 ['소녀', '팬', 'is', '뭔들', '모든'],
 ['소녀', '꽃길을', '걷다', '마주친', 'des'],
 ['소녀', '아무', '느낌', '없었는데', '하루'],
 ['소녀', '마냥', '막', '쿵쿵대', '숨길']]

In [227]:
unique(next_word(male_t, '소녀'))

[['소녀', '야', '너만을', '사랑해', '난'],
 ['소녀', '낯설지가', '않아', '아하', '꿈속에서'],
 ['소녀', '불러', '이', '밤을', '따뜻하게'],
 ['소녀', '마치', '거짓말처럼', '돌아와줘', '왔다'],
 ['소녀', '왠지', '그', '옆에서', '놀렸었던'],
 ['소녀', '왠지', '흘겨보며', '웃고', '있던'],
 ['소녀', 'OK', '인기가', '많은', '너'],
 ['소녀', 'hands', 'up', 'All', 'my'],
 ['소녀', '그', '그', '그', '입술을'],
 ['소녀', '내', '심장박동', '조절이', '안']]

In [224]:
unique(next_word(female_t, '소년'))

[['소년', '내게', '다가와', '수줍게', '말했잖아'], ['소년', '소녀', '꽃길을', '걷다', '마주친']]

In [154]:
unique(before_next_word(female_t, '소녀'))

[['것도', '너무', '많은', '꿈을', '갖고있는', '소녀', '일', '뿐이야', '너', '정신차려'],
 ['하지는', '말아줘', '나', '부탁할게', '어린', '소녀', '꿈을', '짓밟진', '말아', '줘'],
 ['어지럽게', '해', '나', '너만의', '영원한', '소녀', '팬', 'is', '뭔들', '모든'],
 ['버릴것', '같아', '난', '정말', '사랑스런', '소녀', '당신이', '몰라줄', '뿐', '그대가'],
 ['더', '크게', 'Yes', 'Right', '때로는', '소녀', '같게', '하나하나', '조심스럽게', '행동하기'],
 ['조금', '작은키에', '긴', '머리', '날리는', '소녀', '커다란', '그', '눈에', '내'],
 ['잊지', '않고', '머릴', '빗던', '꿈꾸던', '소녀', '행복한', '그', '때로', '그'],
 ['또', '이러지', '내', '마음', '열여덟', '소녀', '마냥', '막', '쿵쿵대', '숨길'],
 ['오빠', '앞에', '서면', '난', '순진한', '소녀', '아무', '느낌', '없었는데', '하루'],
 ['소녀가', 'Tell', 'me', 'why', '소년', '소녀', '꽃길을', '걷다', '마주친', 'des'],
 ['꿈만', '같아', '꿈만', '같아', '소년', '소녀', '꽃길을', '걷다', '마주친', 'des']]

In [155]:
unique(before_next_word(male_t, '소녀'))

[['모두가', '오늘만은', '외롭지', '않게', '성냥팔이', '소녀', '불러', '이', '밤을', '따뜻하게'],
 ['put', 'your', 'hands', 'up', '세기', '소녀', 'hands', 'up', 'All', 'my'],
 ['옛날', '작은', '눈이', '정말', '귀여웠던', '소녀', '왠지', '그', '옆에서', '놀렸었던'],
 ['옛날', '둥근', '볼이', '붉게', '수줍었던', '소녀', '왠지', '흘겨보며', '웃고', '있던'],
 ['버', '버', '버블껌처럼', '수줍은', '한', '소녀', 'OK', '인기가', '많은', '너'],
 ['볼', '빨개진', '널', '보면', '수줍은', '소녀', '내', '심장박동', '조절이', '안'],
 ['참', '확고한데요', '딱', '여리', '여리하고', '소녀', '감성', '가진', '찌질이가', '나라'],
 ['닮아서', '일거야', '소녀야', '영원히', '사랑해', '소녀', '야', '너만을', '사랑해', '난'],
 ['시작한다면', '말리진', '않을', '테지만', '작은', '소녀', '그', '그', '그', '입술을'],
 ['자연재해', '모르겠어', '머리는', '뱅뱅', '작은', '소녀', '그', '그', '그', '입술을'],
 ['아주', '잠깐', '왔다가', '떠난', '그', '소녀', '마치', '거짓말처럼', '돌아와줘', '왔다'],
 ['걷는', '여자애들', '교복', '단정한', '긴머리', '소녀', '낯설지가', '않아', '아하', '꿈속에서']]

In [156]:
unique(before_next_word(female_t, '소년'))

[['않아', '난', '전에', '말했던', '그', '소년', '내게', '다가와', '수줍게', '말했잖아'],
 ['싶은', '소녀가', 'Tell', 'me', 'why', '소년', '소녀', '꽃길을', '걷다', '마주친'],
 ['나', '꿈만', '같아', '꿈만', '같아', '소년', '소녀', '꽃길을', '걷다', '마주친']]

In [153]:
unique(before_next_word(male_t, '소년'))

[['서성거리던', '그때', '철부지', '열아홉살', '코흘리게', '소년', '다시', '그자리로', '돌아가려', '해도'],
 ['위에', '한숨이', '고여', '현실이', '두려운', '소년', '공을', '던질', '때면', '유일하게'],
 ['소녀', '왠지', '그', '옆에서', '놀렸었던', '소년', '전학', '가며', '첨으로', '맘이'],
 ['소녀', '왠지', '흘겨보며', '웃고', '있던', '소년', '사진', '속에', '멈춰서', '그때를'],
 ['탈의해', '어수선하게', '벙찌지', '말아', '양치기', '소년', '같이', '사방을', '전전하며', 'Blah'],
 ['버리고', '마냥', '미소만', '짓던', '그런', '소년', '말고', '난', '다시', '태어났어'],
 ['없는', '바달', '항해할', '꿈을', '꾼', '소년', '같았어', '비를', '맞고', '떨려'],
 ['해도', '흘러내려', '모래알처럼', '난', '철부지', '소년', '부정하네', '악바리처럼', '아니라', '해도'],
 ['문을', '열어', 'Rap', '그래', '빛의', '소년', '넌', '할', '수', '있어']]

In [45]:
unique(before_word(female_t, '되어줘'))

[['OH', '슈퍼히어로가', '되어줘'],
 ['OH', '빛이', '되어줘'],
 ['있는', '연인이', '되어줘'],
 ['비밀노래', '열쇠가', '되어줘'],
 ['날', '사랑하게', '되어줘'],
 ['나에게', '추억만', '되어줘'],
 ['나만의', '사랑이', '되어줘'],
 ['나의', '별이', '되어줘'],
 ['친구가', '항상', '되어줘'],
 ['내', '사랑이', '되어줘']]

In [46]:
unique(before_word(male_t, '되어줘'))

[['퍼즐이', '제발', '되어줘'],
 ['저', '달이', '되어줘'],
 ['U', '빛이', '되어줘'],
 ['아름다운', '꿈이', '되어줘'],
 ['나만의', '것', '되어줘'],
 ['어두워', '빛이', '되어줘'],
 ['내게', '우산이', '되어줘'],
 ['내', '짝이', '되어줘'],
 ['밝혀', '빛이', '되어줘'],
 ['못하게', '빛이', '되어줘'],
 ['baby', '빛이', '되어줘'],
 ['oh', '빛이', '되어줘'],
 ['피어나', '빛이', '되어줘'],
 ['e', '빛이', '되어줘'],
 ['없어', '빛이', '되어줘'],
 ['자연스럽게', '리드해', '되어줘'],
 ['나의', '춤이', '되어줘'],
 ['밤하늘에', '빛이', '되어줘'],
 ['빛나는', '별이', '되어줘'],
 ['내게', '빛이', '되어줘'],
 ['영원한', '빛이', '되어줘'],
 ['내', '사랑이', '되어줘'],
 ['마지막', '욕심이', '되어줘'],
 ['밤', '빛이', '되어줘'],
 ['내', '행운이', '되어줘'],
 ['내', '반쪽이', '되어줘'],
 ['내', '여자가', '되어줘'],
 ['ice', 'ice', '되어줘']]

In [265]:
unique(before_word(female_t, '해줘'))

[['사랑', '잊게', '해줘'],
 ['친구처럼', '따뜻하게', '해줘'],
 ['솔직히', '말을', '해줘'],
 ['나를', '생각좀', '해줘'],
 ['생각', '좀', '해줘'],
 ['널', '잊지않게', '해줘'],
 ['널', '떠나게', '해줘'],
 ['사랑했다', '말을', '해줘'],
 ['꼭', '말을', '해줘'],
 ['날', '떠나게', '해줘'],
 ['얘기', '꼭', '해줘'],
 ['웃으면서', '말을', '해줘'],
 ['늘', '함께', '해줘'],
 ['한', '마디만', '해줘'],
 ['언제까지나', '함께', '해줘'],
 ['수', '있게', '해줘'],
 ['제발', '꿈이라', '해줘'],
 ['내게', '말을', '해줘'],
 ['닦고', '행복', '해줘'],
 ['날', '행복하게', '해줘'],
 ['나와', '함께', '해줘'],
 ['나를', '뜨겁게', '해줘'],
 ['나를', '느끼게', '해줘'],
 ['심하게', '말을', '해줘'],
 ['날', '벗어나게', '해줘'],
 ['내게', '고백을', '해줘'],
 ['가슴이', '말을', '해줘'],
 ['잃지', '않게', '해줘'],
 ['안에', '있게', '해줘'],
 ['잊었나', '말을', '해줘'],
 ['진심이', '아니라', '해줘'],
 ['좀더', '말을', '해줘'],
 ['기쁘게', '기쁘게', '해줘'],
 ['느끼게', '느끼게', '해줘'],
 ['뭐든', '말을', '해줘'],
 ['어서', '말을', '해줘'],
 ['살며시', 'kiss', '해줘'],
 ['나를', '설레게', '해줘'],
 ['꺼', '라고', '해줘'],
 ['sorry', '확실히', '해줘'],
 ['No', '확실히', '해줘'],
 ['말', '좀', '해줘'],
 ['난', '모르게', '해줘'],
 ['잊고싶어', '어떻게', '해줘'],
 ['아픔을', '감추게', '해줘'],
 ['나', 'kiss', '해줘']

In [263]:
unique(before_word(male_t, '해줘'))

[['여기서', '쉬게', '해줘'],
 ['없었던', '일로', '해줘'],
 ['내게', '거짓말이라도', '해줘'],
 ['나를', '있게', '해줘'],
 ['나를', '기쁘게만', '해줘'],
 ['결심했다면', '말을', '해줘'],
 ['숨을', '쉴수없게', '해줘'],
 ['어떤', '얘기라도', '해줘'],
 ['안에', '간직', '해줘'],
 ['내게', '전화를', '해줘'],
 ['나란걸', '믿게', '해줘'],
 ['고통받지', '않게', '해줘'],
 ['되도', '모른다', '해줘'],
 ['오히려', '미안하게', '해줘'],
 ['영원히', '함께', '해줘'],
 ['나', '울지않게', '해줘'],
 ['많이', '사랑좀', '해줘'],
 ['사랑', '좀', '해줘'],
 ['한마디만', '내게', '해줘'],
 ['끝을', '벗어나게', '해줘'],
 ['내게', '말을', '해줘'],
 ['얼굴을', '잊게', '해줘'],
 ['내가', '믿게만', '해줘'],
 ['서', '있겠다', '해줘'],
 ['여기서', '같이', '해줘'],
 ['끝나지', '않게', '해줘'],
 ['여기', '서있겠다', '해줘'],
 ['같이', '있게', '해줘'],
 ['나에게', '말을', '해줘'],
 ['세상에', '있기만', '해줘'],
 ['솔직하게', '말을', '해줘'],
 ['보인다고', '말을', '해줘'],
 ['좀', '어떻게', '해줘'],
 ['차라리', '차갑게', '해줘'],
 ['알지', '못하게', '해줘'],
 ['늘', '뿌려놓게', '해줘'],
 ['수', '있게', '해줘'],
 ['보고', '말을', '해줘'],
 ['볼수', '있게', '해줘'],
 ['돌아와', 'Kiss', '해줘'],
 ['더', '말을', '해줘'],
 ['사랑을', '갖게', '해줘'],
 ['깊이', '함께', '해줘'],
 ['한', '번만이라도', '해줘'],
 ['흔들리지', '않게만', '해줘'],
 [

In [274]:
unique(next_word(female_t, '떨려'))

[['떨려', '그럴', '수', '없어', '넌'],
 ['떨려', '오기에', '눈물에', '기대', '늘'],
 ['떨려', '끌려', '첫눈에', '난', '반했어'],
 ['떨려', '너무', '우쭐하지마', '재미없으면', '어디로'],
 ['떨려', '맘이', '자꾸', '떨려', '왠지'],
 ['떨려', '왠지', '좋은일이', '올', '것'],
 ['떨려', '살짝', '뜨는', '긴장마저', '내겐'],
 ['떨려', '이렇게', 'Get', 'Up', '한'],
 ['떨려', '내', '눈이', '흔들려', 'Wanna'],
 ['떨려', '웃음이나', '우', '언제나', '너를'],
 ['떨려', '웃음이나', '우', '다가와', '비가'],
 ['떨려', '심장에', '신호가', '그만큼', '널'],
 ['떨려', '모른척', '하고', '난', '싶어도'],
 ['떨려', '모른척', '하고만', '싶어도', '눈에'],
 ['떨려', '냉면', '냉면', '냉면', '질겨도'],
 ['떨려', 'Gee', 'Gee', 'Gee', 'Gee'],
 ['떨려', '그래도', '될까', '기다려볼까', 'Oh'],
 ['떨려', '따뜻한', '너의', '품에', '안기고만'],
 ['떨려', 'we', 'got', 'stay', 'together'],
 ['떨려', 'I', 'love', 'you', 'forever'],
 ['떨려', 'Nothing', 'is', 'everything', 'It'],
 ['떨려', 'Nothing', 'is', 'everything', '나의'],
 ['떨려', 'Nothing', 'is', 'everything', '기분'],
 ['떨려', '너도', '똑같니', 'Chemical', 'X'],
 ['떨려', '꿈에도', '넌', '모를걸', '공기마저'],
 ['떨려', 'Oh', 'So', 'Stylis', 'h'],
 ['떨려', '말도', '못했어', 'Whoo', 'What'],
 [

In [58]:
unique(next_word(female_t, '아이처럼'))

[['아이처럼', 'Hello', 'Hello', 'Hello', 'Hello'],
 ['아이처럼', '들뜨고', '라랄라라랄라라', '새콤달콤', 'yeah'],
 ['아이처럼', '날', '감싸는', 'Blanket', '흐릿해진'],
 ['아이처럼', '너무', '좋아서', '방울방울', '추억이'],
 ['아이처럼', '그만', '징징대', '날벼락', '맞은거니'],
 ['아이처럼', 'need', 'you', '그래서', 'without'],
 ['아이처럼', '괴롭히곤', 'Oops', '차원', '생각으로'],
 ['아이처럼', '아무것도', '놓지', '못해요', '내'],
 ['아이처럼', '난', '모든', '걸', '잃은'],
 ['아이처럼', '웃는', '얼굴', '가끔', '가끔씩'],
 ['아이처럼', '헤매는', '건', '당연해', '이'],
 ['아이처럼', '행복해', '상상했던', '사랑에', '빠진'],
 ['아이처럼', '안기고', '싶어', '네', '품에서'],
 ['아이처럼', '맘이', '급해', '네', '답을'],
 ['아이처럼', '웃게', '해줘', '고마워', '사랑한다'],
 ['아이처럼', 'come', 'on', '쉽게', '뜨겁고'],
 ['아이처럼', '사랑할거야', '남은', '일', '하루'],
 ['아이처럼', '날', '어쩜', '별나고', '참'],
 ['아이처럼', '기뻐해', '줄까', '너무나', '궁금해져요'],
 ['아이처럼', '텅빈', '거리를', '헤매죠', '그대'],
 ['아이처럼', '동화속에', '나오는', '그날처럼', '별이'],
 ['아이처럼', '가끔은', '겁이', '났어', '돌아갈'],
 ['아이처럼', '순수한', '내', '사랑', '짧았던'],
 ['아이처럼', '아이처럼', '꿈을', '꾸게', '해'],
 ['아이처럼', '꿈을', '꾸게', '해', 'Move'],
 ['아이처럼', '막', '웃게', '만들어', 'Come'],
 ['아이처럼

In [275]:
unique(next_word(male_t, '떨려'))

[['떨려', '오는', '걸', '아무리', '태연한'],
 ['떨려', '자존심', '상해', '단', '한번도'],
 ['떨려', '와도', '난', '내일의', '태양과'],
 ['떨려', '오는걸', '느껴', 'All', 'eyes'],
 ['떨려', '예전', '그', '시간으로', '돌아가고'],
 ['떨려', 'the', 'clos', 'er', 'you'],
 ['떨려', '하던', '나의', '모습을', '어느새'],
 ['떨려', '사귈때는', '나를', '그렇게', '못살게'],
 ['떨려', '돌아서', '버렸어', '내앞에', '다시'],
 ['떨려', '와', '빈틈을', '보인', '그'],
 ['떨려', '상상한', '것처럼', '우린', '잘'],
 ['떨려', 'Gee', 'Gee', 'Gee', 'Gee'],
 ['떨려', '나', '벌써', '아무', '말'],
 ['떨려', '하나', '둘', '셋', '세면'],
 ['떨려', 'cuz', 'i', 'm', 'a'],
 ['떨려', '소중한', '넌', '바로', '나의'],
 ['떨려', '사실은', '몇', '번씩', '이런'],
 ['떨려', '이런', '날', '안아줘', 'cuz'],
 ['떨려', '숨이', '막혀', 'Baby', '나라면'],
 ['떨려', '쿵쾅거리는', '내', '심장과', '맥박수부터'],
 ['떨려', '이밤이', '가기', '전에', '너에게'],
 ['떨려', 'Today', '좋은', '일이', '일어날것'],
 ['떨려', 'Hello', 'my', 'ex', 'ex'],
 ['떨려', '너무', '너무', '너무', '너무'],
 ['떨려', '잠', '못', '이뤘죠', '이'],
 ['떨려', '어떡하니', '이', '맘', '아마'],
 ['떨려', '알아', '매일', '같던', '하루가'],
 ['떨려', '왜이래', 'Blackout', 'Blackout', 'Blackout'],
 ['떨려', '왜이래', 'I

In [285]:
unique(next_word(female_t, '자존심'))

[['자존심', '있잖아', 'Show', 'me', 'love'],
 ['자존심', '그래', '널', '대신해', '줄'],
 ['자존심', '때문은', '아니야', '한번이라도', '내'],
 ['자존심', '때문은', '아니야', '한번이라도', '내앞에서'],
 ['자존심', '걸고', 'Go', 'this', 'is'],
 ['자존심', '모두', '잊고', '이렇게', '괴로워도'],
 ['자존심', '허락할', '수', '없어', 'oh'],
 ['자존심', '때문에', '항상', '망설이던', '내가'],
 ['자존심', '강한', '내가', '변했다고', '놀리지'],
 ['자존심', '센', '내가', '변해가죠', '어느새'],
 ['자존심', '짓밟아놓고', '내', '맘을', '찢어놓고'],
 ['자존심', '다', '버리고', '줬는데', '참'],
 ['자존심', '다', '버리고', '미친척', '널'],
 ['자존심', '다', '버리고', '다시', '붙잡을까'],
 ['자존심', '세우다', '못해', '맘에도', '없는'],
 ['자존심', '따윈', '다', '버렸어', '우스워'],
 ['자존심', '따윈', '아무', '상관', '없는걸'],
 ['자존심', '다', '뺏어줄래', '니', '날카로운'],
 ['자존심', '버리고', '널', '용서했다면', '이렇게'],
 ['자존심', '따위는', '집에', '두고', '와주기'],
 ['자존심', '따윈', '없어', '위험해', '하지만'],
 ['자존심', '탓', '때문에', '말도', '못해'],
 ['자존심', '다', '버리면', '다시', '내게'],
 ['자존심', '때문에', '미안해', '한마디', '못해'],
 ['자존심', '버리고서', '말할게요', '저녁', '해가'],
 ['자존심', '난', '참', '예쁘다', '정말로'],
 ['자존심', '건드려', '건드려', '시퍼렇게', '멍들어'],
 ['자존심', '같은', '것', '다

In [286]:
unique(next_word(male_t, '자존심'))

[['자존심', '다', '구겨', '버리신건가', '결국'],
 ['자존심', '땜에', '화만', '내나봐', '니'],
 ['자존심', '싸움인걸', '특별한', '것도', '없고'],
 ['자존심', 'Yeah', 'Baby', 'Give', 'my'],
 ['자존심', '상해', '단', '한번도', '헌팅은'],
 ['자존심', '팍팍', '무너졌어', '지금', '내가'],
 ['자존심', '버릴땐', '버려야잖아', '이젠', '눈을'],
 ['자존심', '천년의', '무궁화가', '넝쿨', '땜에'],
 ['자존심', '맘', '지키며', '눈물', '따윈'],
 ['자존심', '따위는', '접어서', '구긴', '채'],
 ['자존심', '많던', '옛날이', '생각', '나'],
 ['자존심', '아직도', '거만한걸', '나를', '보는'],
 ['자존심', '다시', '예전같이', '세울', '수는'],
 ['자존심', '결국', '나이살에', '죽게될', '걸'],
 ['자존심', '버려졌음', '한숨', '하하하하하', '하'],
 ['자존심', '많았던', '너무도', '맘이', '가난했던'],
 ['자존심', '니', '생각만해선', '세상살기', '힘들어'],
 ['자존심', '따윈', '고이', '접어', '저'],
 ['자존심', '너의', '웃는', '얼굴', '앞에서'],
 ['자존심', '강한', '남자다운', '터프가이', '근데'],
 ['자존심', '세우다', '못해', '맘에도', '없는'],
 ['자존심', '때문에', '지금', '내', '말'],
 ['자존심', '때문에', 'sorry', 'but', 'I'],
 ['자존심', '때문에', '뒤돌아서면', '또', '다시'],
 ['자존심', '앞세워', '널', '힘들게', '한'],
 ['자존심', '같은건', '넌', '부릴게', '아냐'],
 ['자존심', '뿐이었고', '침묵', '속의', '그'],
 ['자존심', '부렸던', '나', '자존심'

In [296]:
unique(next_word(female_t, '여자는'))

[['여자는', '모두', '너의', '뜻대로', '너무'],
 ['여자는', '보기', '힘들거라고', '인연은', '그렇게'],
 ['여자는', '더', '볼수', '없을거라고', '말했지만'],
 ['여자는', '오직', '하나의', '사랑만을', '담을'],
 ['여자는', '여자답게', '남자라면', '리드해', '남자답게'],
 ['여자는', '이젠', '없지', '평소와', '달리'],
 ['여자는', '너무', '많아', '이', '손'],
 ['여자는', '너무', '많아', 'Oh', 'oh'],
 ['여자는', '너무', '많아', 'Baby', 'don'],
 ['여자는', '그토록', '날', '원하고', '있는'],
 ['여자는', '없어', '알겠어', '그럼', '이제'],
 ['여자는', '계속', '변하거든', '멈추지', '않아'],
 ['여자는', '말야', '우우우우', '나처럼', '말야'],
 ['여자는', '자존심이라고', '그래서', '네게도', '지켜보려'],
 ['여자는', '갈대', '같은', '거야', '사랑은'],
 ['여자는', '난', '될', '수', '없어'],
 ['여자는', 'good', 'bye', '멘트는', '필요'],
 ['여자는', '이러면', '안', '되는데', 'Ooh'],
 ['여자는', '뻔해', '네가', '더', '뻔해'],
 ['여자는', '알고', '있니', '기도', '안차'],
 ['여자는', '사랑을', '먹고', '살아', '가는'],
 ['여자는', '세상에', '없어', '그래서', '이런'],
 ['여자는', '세상에없어', '그래서', '이런너를', '사랑해'],
 ['여자는', '바람둥이야', '왜', '그', '걸'],
 ['여자는', '남자가', '많아', '왜', '혼자'],
 ['여자는', '바람둥이야', '왜', '그걸', '모르죠'],
 ['여자는', '없잖아', 'Ah', '눈길', '한번'],
 ['여자는', '다', '원해', '진짜', '

In [297]:
unique(next_word(male_t, '여자는'))

[['여자는', 'kiss', 'of', 'fire', '한여자는'],
 ['여자는', '목이긴', '모딜리아니야', '그동안', '한번도'],
 ['여자는', '과연', '어떤', '여잘까', '이제야'],
 ['여자는', '그', '만큼의', '가치가', '있어'],
 ['여자는', '많다', '꼭', '그녀일', '필요는'],
 ['여자는', '너무', '어려운', '걸', '사랑을'],
 ['여자는', '기쁨', '아니면', '슬픔', '다섯'],
 ['여자는', '감정이', '풍부한', '사람이지', '조그만'],
 ['여자는', '진짜', '무서운', '여자이지', '남자들은'],
 ['여자는', '없는지', '이', '외로운', '밤'],
 ['여자는', '여기', '모두', '모였어', '나는'],
 ['여자는', '안될까', 'yeah', '너는', '조금씩'],
 ['여자는', '여기모두', '모였어', '나는', '정신없이'],
 ['여자는', '왜', '내게', '이렇게도', '지독한'],
 ['여자는', '많아', '떠나는', '너를', '보며'],
 ['여자는', '지성과', '미모', 'you', 'got'],
 ['여자는', '많아', '화려했던', '지난날', '다시'],
 ['여자는', '많다나', '하하하', '하고', '한번'],
 ['여자는', '니', '얼굴보고', '확', '갔고'],
 ['여자는', '마지막', '사랑을', '둘다', '할수'],
 ['여자는', '역시', '예뻐야돼', '한동안', '즐거웠지'],
 ['여자는', '세상엔', '없다', 'I', 'got'],
 ['여자는', '뭐니', '뭐니', '해도', '착한'],
 ['여자는', '다', '여자지', '별', '의미'],
 ['여자는', '끝까지', '지켜', '내', '사랑을'],
 ['여자는', '사랑', '안', '해', 'I'],
 ['여자는', '쳐다본', '적', '없는', '내가'],
 ['여자는', '없어요', '푹', '빠졌나',

In [137]:
unique(next_word(female_t, '남자는'))

[['남자는', '많고', '많단다', '애가', '왜'],
 ['남자는', '많아', '재빠르게', '날', '흔들어'],
 ['남자는', '내', '남자', '너무', '너무'],
 ['남자는', '조금', '달라', '듣는', '순간'],
 ['남자는', '아냐', '물론', '그렇지만', '마음'],
 ['남자는', '아냐', '내가', '아무리', '네'],
 ['남자는', '되지', '말란', '말이야', 'Do'],
 ['남자는', '다', '그래', '아빠', '말처럼'],
 ['남자는', '다', '늑대', '거기', '못난이'],
 ['남자는', '많았대두', '그냥', '웃기만', '하는'],
 ['남자는', '사랑할', '자격', '없잖아', '이별을'],
 ['남자는', '내겐', '필요없어', '갑자기', '울려대는'],
 ['남자는', '없어', '남자는', '우우', '우'],
 ['남자는', '우우', '우', '우우우', '우우'],
 ['남자는', '모두', '똑같다는', '말', '그렇게'],
 ['남자는', '맘에', '차지도', '않았던', '난데'],
 ['남자는', '싫어', '싫어', '난', '모든'],
 ['남자는', '없어', '모든게', '아름답게', '보여'],
 ['남자는', '첨이야', '내', '맘을', '설레게'],
 ['남자는', '바로', '너', '어리숙해', '보여도'],
 ['남자는', '지겨워', '웃기기는', '하지만', '촐싹대고'],
 ['남자는', '많아', '많아', '많아', '넌'],
 ['남자는', '여자맘을', '몰라요', '남자는', '그렇게'],
 ['남자는', '그렇게', '떠나가네요', '내맘', '다'],
 ['남자는', '세상에', '많고', '많겠지만', '너같이'],
 ['남자는', '세상에', '진짜', '많겠지만', '내가'],
 ['남자는', '왜', '여자친구가', '있는', '거야'],
 ['남자는', '여자가', '좋아한다', '혼자', '착각만']

In [139]:
unique(next_word(female_t, '남자는'))

[['남자는', '많고', '많단다', '애가', '왜'],
 ['남자는', '많아', '재빠르게', '날', '흔들어'],
 ['남자는', '내', '남자', '너무', '너무'],
 ['남자는', '조금', '달라', '듣는', '순간'],
 ['남자는', '아냐', '물론', '그렇지만', '마음'],
 ['남자는', '아냐', '내가', '아무리', '네'],
 ['남자는', '되지', '말란', '말이야', 'Do'],
 ['남자는', '다', '그래', '아빠', '말처럼'],
 ['남자는', '다', '늑대', '거기', '못난이'],
 ['남자는', '많았대두', '그냥', '웃기만', '하는'],
 ['남자는', '사랑할', '자격', '없잖아', '이별을'],
 ['남자는', '내겐', '필요없어', '갑자기', '울려대는'],
 ['남자는', '없어', '남자는', '우우', '우'],
 ['남자는', '우우', '우', '우우우', '우우'],
 ['남자는', '모두', '똑같다는', '말', '그렇게'],
 ['남자는', '맘에', '차지도', '않았던', '난데'],
 ['남자는', '싫어', '싫어', '난', '모든'],
 ['남자는', '없어', '모든게', '아름답게', '보여'],
 ['남자는', '첨이야', '내', '맘을', '설레게'],
 ['남자는', '바로', '너', '어리숙해', '보여도'],
 ['남자는', '지겨워', '웃기기는', '하지만', '촐싹대고'],
 ['남자는', '많아', '많아', '많아', '넌'],
 ['남자는', '여자맘을', '몰라요', '남자는', '그렇게'],
 ['남자는', '그렇게', '떠나가네요', '내맘', '다'],
 ['남자는', '세상에', '많고', '많겠지만', '너같이'],
 ['남자는', '세상에', '진짜', '많겠지만', '내가'],
 ['남자는', '왜', '여자친구가', '있는', '거야'],
 ['남자는', '여자가', '좋아한다', '혼자', '착각만']

In [303]:
unique(next_word(male_t, '남자는'))

[['남자는', '관심', '있는', '여자에게', '톡톡'],
 ['남자는', '뭐하십니까', '스폰서는', '아니십니까', '가슴이'],
 ['남자는', '내가', '아냐', '제발', '미련따윈'],
 ['남자는', '그래도', '괜찮단', '친구의', '바보같은'],
 ['남자는', '아니겟죠', '선택', '무조건', '상대와는'],
 ['남자는', '지금', '여기', '없어', '이'],
 ['남자는', '지금', '여기', '없어', '여기없어'],
 ['남자는', '지금', '여기', '없어', '작업'],
 ['남자는', '아니야', '니가', '쉽게', '보인단'],
 ['남자는', '그녀를', '잃고', '나서야', '느끼게'],
 ['남자는', '아니지만', '너무', '날', '오래'],
 ['남자는', '너를', '잘', '몰라', '너를'],
 ['남자는', '주머니꽉', '차', '시계는', '안봐멋진'],
 ['남자는', '보나마나', '날라릴텐데', '하하', '이런'],
 ['남자는', '용기와', '돈', 'and', 'I'],
 ['남자는', '바로', '내가', '되야해', '너를'],
 ['남자는', '바로', '내가', '되야', '해'],
 ['남자는', '울면', '안되는', '거야', '왜'],
 ['남자는', '울면', '안돼는', '거야', '왜'],
 ['남자는', '울면', '안돼는거야', '왜', '왜'],
 ['남자는', '너에겐', '없다', 'bye', 'bye'],
 ['남자는', '바로', '나란걸', '날', '원하고'],
 ['남자는', 'TV', '로', '봤겠지만', '난'],
 ['남자는', '노래', '부르지만', '눈물이', '나는'],
 ['남자는', '입보다', '행동이', '먼저', '되'],
 ['남자는', '나야', '너의', '입술', '위에'],
 ['남자는', '짐승', '짐승', '짐승이라', '남자는'],
 ['남자는', '짐승이라', '말할', '때', '더']

In [299]:
unique(next_word(female_t, '여자니까'))

[['여자니까', '그래요', '푹', '빠졌죠', '사랑에'],
 ['여자니까', '허둥거려', '난', '늘', '허둥거려'],
 ['여자니까', 'Yeah', 'I', 'm', 'Jus'],
 ['여자니까', '감출', '수', '없는', '시선'],
 ['여자니까', 'falling', 'in', 'love', 'falling'],
 ['여자니까', '이해해주길', '속', '마음', '들킬']]

In [302]:
unique(next_word(female_t, '남자니까'))

[['남자니까', '내게만', '있어', '딴', '생각은'],
 ['남자니까', 'Get', 'out', '영원히', 'the'],
 ['남자니까', '나', '하나만', '좋으면', '돼'],
 ['남자니까', '그', '누구도', '안', '줄래'],
 ['남자니까', '남자답게', '당당하게', '말할게', '내'],
 ['남자니까', '다', '지난', '일이지만', '너가'],
 ['남자니까', '나', '좋단', '남잔', '많으니까']]

In [298]:
unique(next_word(male_t, '여자니까'))

[['여자니까', '안돼', '보기', '싫어', '안돼'],
 ['여자니까', '단지', '널', '걱정하는', '것'],
 ['여자니까', 'Three', 'time', '정해두지', '않았어'],
 ['여자니까', '다른', '남자가', '널', '쳐다볼'],
 ['여자니까', '안', '그런척하며', '너의', '짧은'],
 ['여자니까', '제발', '제발', '야하게', '입지마'],
 ['여자니까', '나', '아닌', '남자는', '쳐다도'],
 ['여자니까', '너만', '몰랐을', '뿐', '이미'],
 ['여자니까', 'So', 'Let', 'me', 'be'],
 ['여자니까', '누난나', '누난나', '왜', '왜몰라'],
 ['여자니까', '못', '본', '척', '하지마'],
 ['여자니까', '누나', '너', '말야', '날'],
 ['여자니까', '그', '자리에서', '기다려', '내'],
 ['여자니까', 'ye', 'ye', '내가', '나쁜'],
 ['여자니까', '쓰레기', '같은', '성격도', '보잘것없는'],
 ['여자니까', '쓰레기', '같은', '성격도', '보잘']]

In [300]:
unique(next_word(male_t, '남자니까'))

[['남자니까', '가진', '게', '없는', '게'],
 ['남자니까', '아직도', '널', '잊지못하고', '속으론'],
 ['남자니까', '우리는', '항상', '서로', '곁에서'],
 ['남자니까', '너만', '보면', '둥둥둥둥둥', '미쳐버린'],
 ['남자니까', '한숨은', '바람이', '되어', '추억까지'],
 ['남자니까', '가슴이', '메어져도', '하늘아', '제발'],
 ['남자니까', '넘어가지', '가슴을', '두드리며', '참고'],
 ['남자니까', '미친', '거', '아니니', '솟구쳐'],
 ['남자니까', '넘어가지', '많이', '아팠어', '자신을'],
 ['남자니까', '나는', '이런', '남자니까', '참을래'],
 ['남자니까', '참을래', '비틀', '비틀', '비틀'],
 ['남자니까', '변한', '너', '때문이니까', '한숨'],
 ['남자니까', '남자답게', '당당하게', '말할게', '내'],
 ['남자니까', '안', '듣는', '척하며', '너의'],
 ['남자니까', '다', 'pass', '요', 'Rock'],
 ['남자니까', '팍팍', '밀어붙여', '딴', '놈이'],
 ['남자니까', '눈이', '가', '자꾸', '눈이']]

In [287]:
unique(next_word(female_t, '여자답게'))

[['여자답게', '남자라면', '리드해', '남자답게', '삼팔선을'],
 ['여자답게', '왜', '나를', '갖고', '장난쳐'],
 ['여자답게', '아냐', '내숭은', '어색해', '털털하면']]

In [294]:
unique(next_word(female_t, '남자답게'))

[['남자답게', '먼저', '이끌어줄', '니', '모습'],
 ['남자답게', '멋있게', '잊어야만해', '잊어야만', '해'],
 ['남자답게', '다가서봐', '혹시', '누가', '알아'],
 ['남자답게', '느껴봐', '내가', '그동안', '견뎌온'],
 ['남자답게', '맹세한', '것도', '거짓말', '너'],
 ['남자답게', '지독한', '사랑을', '해봐', 'yeah'],
 ['남자답게', '삼팔선을', '긋는', '건', '여자들의'],
 ['남자답게', '용기를', '내요', '이', '순간을'],
 ['남자답게', 'Shoot', 'it', 'out', '왜'],
 ['남자답게', '안아줘요', '사랑을', '얘기해요', '밤새도록'],
 ['남자답게', '말해봐', 'Oh', 'my', 'gos'],
 ['남자답게', '좀', '더', '개성있게', '널'],
 ['남자답게', '좀', '더', '나를', '리드해줘'],
 ['남자답게', '내게', '얘기해줘', '더', '답답하게'],
 ['남자답게', '내게', '얘기해', '더', '답답하게'],
 ['남자답게', '말해주겠니', '후후후', '후우후후', '후후후'],
 ['남자답게', '말해주겠니', 'what', 'you', 'waiting'],
 ['남자답게', '그만', '말을', '해봐', '나를'],
 ['남자답게', '다가와', '꽉', '안아줄래', 'Baby'],
 ['남자답게', '용기', '내봐', 'Say', 'you'],
 ['남자답게', '다신오지', '않을', '기회', '후회하지말고'],
 ['남자답게', '보여줘', '애처럼', '굴지마', '정말'],
 ['남자답게', '당당하게', '말할게', '내', '눈'],
 ['남자답게', '먼저', '이끌어', '달라고', '내게와'],
 ['남자답게', '다가와', '날', '잡아줄래', 'Jus'],
 ['남자답게', '더', '쿨하게', '말해봐', '지금'],
 ['남자답게', '네'

In [295]:
unique(next_word(male_t, '여자답게'))

[['여자답게', '있어달라', '소리치고', '널', '뿌리쳤어']]

In [288]:
unique(next_word(male_t, '남자답게'))

[['남자답게', '사랑에', '치사하지', '말자라며', '하루하루'],
 ['남자답게', '나이를', '먹고', '덧셈과', '뺄셈도'],
 ['남자답게', '잊지마', '세상', '끝에', '있어도'],
 ['남자답게', '보내주지', '못하니', '그래야겠지', '잊어야겠지'],
 ['남자답게', '그녈', '보내줬지', '기가', '막혀'],
 ['남자답게', '그녈', '보내줬지', 'Rap', '기가막혀'],
 ['남자답게', 'show', 'your', 'heart', '더'],
 ['남자답게', '돌아서줄께', '괜찮아', '나는', '너만'],
 ['남자답게', '돌아서줄께', '잊지마', '그가', '널'],
 ['남자답게', '큰소릴쳤지', '하지만', '그녀만', '아는'],
 ['남자답게', '사랑하는', '법', '나도', '그쯤은'],
 ['남자답게', '겁내지', '않고', '기횐', '지금'],
 ['남자답게', '택했어', '한가지', '길', 'jus'],
 ['남자답게', '지킬꺼라', '했는데', '말했는데', '자신있게'],
 ['남자답게', '찢어진', '가슴을', '부둥켜', '안고서'],
 ['남자답게', '말하지', '못', '한', '수줍었던'],
 ['남자답게', '웃는다는', '게', '행여', '다시'],
 ['남자답게', '떠난다', 'Ah', 'Ah', '하나둘셋'],
 ['남자답게', '악수해', '그녈', '내', '품에'],
 ['남자답게', '보낸다', '내', '속에서', '네가'],
 ['남자답게', '맞서겠어', '강해져야', '하고', '단단해야돼'],
 ['남자답게', '맞서겠어', '현실은', '내편이', '아니지만'],
 ['남자답게', '맞서겠어', '인터넷', '보단', '실물이'],
 ['남자답게', '고개를', '들어', '주저앉지', '말고'],
 ['남자답게', '보이려고', '애써', '모르는', '척'],
 ['남자답게', '네게', '말할게', 'Your', 'lif

In [272]:
unique(next_word(female_t, '설레'))

[['설레', '색을', '잃은', '날', '찾지'],
 ['설레', '두근거릴', '때', '달려가면', '나를'],
 ['설레', '이는', '내', '맘', '생각만해도'],
 ['설레', '발레', '제치고', '지쳐', '넘어질때면'],
 ['설레', '난', '깨기', '싫은데', '매일'],
 ['설레', '너무나', '좋아하니까', '제발', '날'],
 ['설레', '온', '맘이', '살짝', '궁금해'],
 ['설레', '설레', '난', '설레었는데', '나만'],
 ['설레', '난', '설레었는데', '나만', '그랬니'],
 ['설레', '니가', '나를', '뺏어줘', '먼저'],
 ['설레', '내', '촉을', '모두', '세워'],
 ['설레', '한걸음씩', '갈래', '지금', '바로'],
 ['설레', '이며', '가슴', '벅차', '그래'],
 ['설레', 'What', 'you', 'gone', 'do'],
 ['설레', '왜', '이러지', '이렇게', '네게'],
 ['설레', '설레는', '이', '맘', '흔들'],
 ['설레', '너의', '목소리', 'oh', 'baby'],
 ['설레', '니', '얼굴좀', '봐', 'oh'],
 ['설레', '어떡해', '너무', '설레', '어떻게'],
 ['설레', '어떻게', '해야해', '시', '분'],
 ['설레', '또', '두근대', '두근대', '두근대'],
 ['설레', '까만', '밤을', '하얗게', '지샌'],
 ['설레', '우리', '사이의', '연결주문', '아로아로'],
 ['설레', '설레', '나는', '막', '설레'],
 ['설레', '나는', '막', '설레', '눈감으면'],
 ['설레', '눈감으면', '기적처럼', '그대가', '내'],
 ['설레', '설레', '아직도', '설레', '내겐'],
 ['설레', '아직도', '설레', '내겐', '너무'],
 ['설레', '내겐', '너무', '소중해서', '아이처럼'],

In [273]:
unique(next_word(male_t, '설레'))

[['설레', '였지', '우리', '그', '여행'],
 ['설레', '못했어', '정말', '많이', '기다렸어'],
 ['설레', '너와의', 'kiss', '그', '순간'],
 ['설레', '혼자', '외로울', '때', '그땐'],
 ['설레', '나', '다운', '내가', '다시'],
 ['설레', '그건', '매일', '매일', '너만을'],
 ['설레', '이러지는', '않는데', '내가', '원래'],
 ['설레', '오', '내가', '왜', '이러니'],
 ['설레', '넌', '여전히', '똑같아', '철'],
 ['설레', 'Hum', '해바라기해바라기', '나의', '향긋함에'],
 ['설레', '오', '점점', '네게', '갈수록'],
 ['설레', '설레', '설레', '나를', '보는'],
 ['설레', '설레', '나를', '보는', '여자맘이야'],
 ['설레', '나를', '보는', '여자맘이야', '너무나'],
 ['설레', '설레', '설레', '내게', '반한'],
 ['설레', '설레', '내게', '반한', '여자맘이야'],
 ['설레', '내게', '반한', '여자맘이야', '거울에'],
 ['설레', '설레', '설레', '나에게', '반한'],
 ['설레', '설레', '나에게', '반한', '여자맘이야'],
 ['설레', '나에게', '반한', '여자맘이야', '얼굴에'],
 ['설레', '설레', '설레', '너를', '보는'],
 ['설레', '설레', '너를', '보는', '남자맘이야'],
 ['설레', '너를', '보는', '남자맘이야', '너무나'],
 ['설레', '설레', '설레', '너에게', '반한'],
 ['설레', '설레', '너에게', '반한', '남자맘이야'],
 ['설레', '너에게', '반한', '남자맘이야', '그러던'],
 ['설레', 'I', 'm', 'gonna', 'make'],
 ['설레', '오래간만에', 'Baby', 'Woh', 'woh'],
 ['설레', '말

In [63]:
unique(next_word(female_t, '해맑은'))

[['해맑은', '얼굴로', '짓궂은', '말투로', '날'],
 ['해맑은', '미소', '어리던', '그', '날엔'],
 ['해맑은', '우리', '둘의', '개구쟁이', '꼬마들과'],
 ['해맑은', '표정이', '사랑을', '타고', '눈부시게'],
 ['해맑은', '아이처럼', '기뻐해', '줄까', '너무나'],
 ['해맑은', '미소', '너무', '진한', '내'],
 ['해맑은', '미소는', '날위한', '사랑인건지', '나는'],
 ['해맑은', '모습', '나의', '하루속에서', '넌'],
 ['해맑은', '웃음만을', '주는', '또', '하나의'],
 ['해맑은', '눈동자에', 'oh', '내가있어', '그대'],
 ['해맑은', '눈동자에', 'Oh', '내가', '있어'],
 ['해맑은', '미소로', '네게', '나', '다가가길']]

In [61]:
unique(next_word(male_t, '해맑은'))

[['해맑은', '너의', '그', '미소가', '나는'],
 ['해맑은', '네', '미소', 'I', 'wanna'],
 ['해맑은', '미소로', '아침을', '열어', '줄'],
 ['해맑은', '미소로', '손을', '내밀어', '내가'],
 ['해맑은', '미소로', '손을', '내밀어', '어두운'],
 ['해맑은', '네', '큰눈에', '반했던게', '어제'],
 ['해맑은', '네', '두', '눈에', '반했던'],
 ['해맑은', '미소', '가득', '담아오는', '내'],
 ['해맑은', '미소', '오늘도', '날', '미치게해'],
 ['해맑은', '눈', '단발머리', '앳된', '모습'],
 ['해맑은', '미소로', '곁에', '있는', '네가'],
 ['해맑은', '웃음으로', '살고싶어', '난', '언제까지나'],
 ['해맑은', '미소로', '나를', '바보로', '만들었소'],
 ['해맑은', '눈빛이', '하늘이', '되고', '끝없이'],
 ['해맑은', '너의', '미소는', '이제', '가슴속에'],
 ['해맑은', '너의', '미소', '잊지', '못한'],
 ['해맑은', '너의', '미소와', '우리', '처음'],
 ['해맑은', '눈', '날바라보는', '그', '보석'],
 ['해맑은', '미소로', '내', '맘', '받아주었죠'],
 ['해맑은', '두눈', '가득', '오직', '나'],
 ['해맑은', '두', '눈', '가득', '오직'],
 ['해맑은', '너의', '웃음', '지켜주고', '싶어'],
 ['해맑은', '모습', '순수한', '모습', '마치'],
 ['해맑은', '모습에', '두근거림', '숨길', '순'],
 ['해맑은', '웃음도', '그녀의', '매력이에요', '그']]

In [85]:
unique(before_word(female_t, '될거야'))

[['oh', '내가', '될거야'],
 ['hey', '내일은', '될거야'],
 ['너는', '후회하게', '될거야'],
 ['도리도리', '잘', '될거야'],
 ['are', '열게', '될거야'],
 ['만나봐', '이해하게', '될거야'],
 ['너도', '알게', '될거야'],
 ['생각이', '나게', '될거야'],
 ['모두', '이뤄지게', '될거야'],
 ['니', '향기가', '될거야'],
 ['멋진', '내가', '될거야'],
 ['언젠간', '알게', '될거야'],
 ['없어', '후회하게', '될거야'],
 ['모든게', '현실이', '될거야'],
 ['너만', '알게', '될거야'],
 ['꿈을', '이루게', '될거야'],
 ['언젠가', '만나게', '될거야'],
 ['아빠에게', '힘이', '될거야'],
 ['한참뒤에', '깨닫게', '될거야'],
 ['다', '잘', '될거야'],
 ['좋은', '하루가', '될거야'],
 ['이제는', '잘', '될거야'],
 ['내', '남자', '될거야'],
 ['여잘', '만나게', '될거야']]

In [83]:
unique(before_word(male_t, '될거야'))

[['모두', '잘', '될거야'],
 ['뭔갈', '알게', '될거야'],
 ['모습을', '보게', '될거야'],
 ['너를', '안게', '될거야'],
 ['넌', '최고가', '될거야'],
 ['너만의', '기사님이', '될거야'],
 ['넌', '놀라게', '될거야'],
 ['듬직한', '남자', '될거야'],
 ['다시', '보게', '될거야'],
 ['너도', '알게', '될거야'],
 ['빛을', '보게', '될거야'],
 ['진실을', '알게', '될거야'],
 ['언젠가는', '미소짓게', '될거야'],
 ['걸', '얻게', '될거야'],
 ['차를', '몰게', '될거야'],
 ['날은', '오늘이', '될거야'],
 ['기도하면', '그렇게', '될거야'],
 ['많다는걸', '알게', '될거야'],
 ['또', '그러게', '될거야'],
 ['이제', '알게', '될거야'],
 ['십만', '점쯤은', '될거야'],
 ['난', '컴퓨터가', '될거야'],
 ['다', '잘', '될거야'],
 ['세상에', '빛이', '될거야'],
 ['더높이', '날게', '될거야'],
 ['함께', '날게', '될거야'],
 ['것도', '알게', '될거야'],
 ['위하는', '기도가', '될거야']]

In [68]:
your = unique(next_word(male_t, '너의'))

In [71]:
your = sum(your, [])

In [75]:
Counter(your).most_common(50)

[('너의', 1754),
 ('마음을', 1),
 ('친구들이', 1),
 ('맘', 1),
 ('모습', 1),
 ('곁엔', 1),
 ('사랑', 1),
 ('환상', 1),
 ('아버님께서', 1),
 ('눈물이', 1),
 ('그', 1),
 ('목소리', 1),
 ('마지막', 1),
 ('집앞에서', 1),
 ('방', 1),
 ('손잡아', 1),
 ('마음', 1),
 ('마음이', 1),
 ('꿈이', 1),
 ('이름', 1),
 ('행복을', 1),
 ('모습이', 1),
 ('환한', 1),
 ('모습을', 1),
 ('눈빛이', 1),
 ('불안한', 1),
 ('달콤한', 1),
 ('한없이', 1),
 ('집', 1),
 ('말을', 1),
 ('말이', 1),
 ('얼굴을', 1),
 ('말도', 1),
 ('키스가', 1),
 ('손길이', 1),
 ('촉촉한', 1),
 ('키스', 1),
 ('키스만', 1),
 ('손길만', 1),
 ('눈으로', 1),
 ('모든게', 1),
 ('맘을', 1),
 ('눈을', 1),
 ('모든', 1),
 ('느낌들이', 1),
 ('얼굴이', 1),
 ('얼굴만', 1),
 ('눈속에', 1),
 ('숨결이', 1),
 ('자릴', 1)]

In [284]:
# 여자아이돌 곡의 가사 중 '(설렘|설레)'가 등장하는 곡 수는?
female_k = female[female['lyrics'].str.contains("자존심", na=False)]
female_k.shape

(66, 18)

In [282]:
# 남자아이돌 곡의 가사 중 '(설렘|설레)'가 등장하는 곡 수는?
male_k = male[male['lyrics'].str.contains("자존심", na=False)]
male_k.shape

(105, 18)

In [269]:
# 여자아이돌 곡의 가사 중 '(설렘|설레)'가 등장하는 곡 수는?
female_k = female[female['lyrics'].str.contains("(설렘|설레)", na=False)]
female_k.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(354, 18)

In [280]:
# 남자아이돌 곡의 가사 중 '(설렘|설레)'가 등장하는 곡 수는?
male_k = male[male['lyrics'].str.contains("(떨림|떨려)", na=False)]
male_k.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(179, 18)

In [261]:
# 여자아이돌 곡의 가사 중 '해줘'가 등장하는 곡 수는?
female_k = female[female['lyrics'].str.contains("해줘", na=False)]
female_k.shape

(465, 18)

In [246]:
# 여자아이돌 곡의 가사 중 '기다려'가 등장하는 곡 수는?
female_k = female[female['lyrics'].str.contains("기다려", na=False)]
female_k.shape

(280, 18)

In [254]:
# 여자아이돌 곡의 가사 중 '기다려'가 등장하는 곡 수는?
female_k = female[female['lyrics'].str.contains("(머리카락|머릿결|입술|엉덩이|허리|다리|몸매)", na=False)]
female_k.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(598, 18)

In [256]:
# 남자아이돌 곡의 가사 중 ' 단어가 등장하는 곡 수는?
male_k = male[male['lyrics'].str.contains("(머리카락|머릿결|입술|엉덩이|허리|다리|몸매)", na=False)]
male_k.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(1042, 18)

In [258]:
all = song[song['lyrics'].str.contains("(머리카락|머릿결|입술|엉덩이|허리|다리|몸매)", na=False)]
all.shape

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(1720, 18)

In [259]:
all = song[song['lyrics'].str.contains("사랑", na=False)]
all.shape

(5375, 18)

In [257]:
male_k.lyrics.tolist()

['항상 난 지나치기만 했네 이젠 난 미련없이 끝내 됐어 찾아냈어 난 모두가 또 나를 원하는가 이젠 난 바라는건 없어 나는 ROCK and RADIO and D A T 나의 기타 BIG ENERGY 나의 장갑속의 너의 큰 반지 눈부신 햇살에 내 모든 것을 맡기고 오래된 그리움은 모두 벗어 제끼고 나의 그 생각들엔 멋진 날개를 달아 이제는 난 지치지 않아 모든걸 난 여태 이겨내왔어 밤새워 내린 하얀눈과 아침을 열어주고 있는 태양 내 맘을 자극하는 바람 난 지금 시작해 KNOCK THE WORLD BREAK THE WALL ROCK and ROLL 하얗게 내린 눈 위로 날아가듯 내 꿈에 큰 날개를 달고서 더 밝은 나의 미래를 약속하듯 세상은 눈부시게 빛나고 있어  길을 떠나며 난 많은 사람들을 지나쳤지 내게는 없던 또 다른 눈빛 시간을 겁내지 말고 이제는 막연히 기다릴수는 없어 오래된 ORGAN소리 하늘을 걸어다니는 다리 눈부신 햇살에 내 모든 것을 맡기고 오래된 그리움은 모두 벗어 제끼고 나의 그 생각들엔 멋진 날개를 달아 이제는 난 지치지 않아 모든걸 난 여태 이겨내왔어 밤새워 내린 하얀눈과 아침을 열어주고 있는 태양 내 맘을 자극하는 바람 난 지금 시작해 KNOCK THE WORLD BREAK THE WALL ROCK and ROLL 하얗게 내린 눈 위로 날아가듯 내 꿈에 큰 날개를 달고서 더 밝은 나의 미래를 약속하듯 세상은 눈부시게 빛나고 있어',
 '난 지금 무엇을 찾으려고 애를 쓰는 걸까 난 지금 어디로 쉬지않고 흘러가는가 난 내 삶의 끝을 본적이 있어 내 가슴속은 갑갑해졌어 내 삶을 막은것은 나의 내일에 대한 두려움 반복됐던 기나긴 날 속에 버려진 내 자신을 본 후 나는 없었어 그리고 또 내일조차 없었어 내겐 점점 더 크게 더해갔던 이 사회를 탓하던 분노가 마침내 증오가 됐어 진실들은 사라졌어 혀 끝에서 YOU MUST COME BACK HOME 떠나간 마음보다 따뜻한 YOU MUST COME BACK HOME 거칠은 인생속에 YOU MUST COM

In [248]:
# 남자아이돌 곡의 가사 중 '기다려'가 단어가 등장하는 곡 수는?
male_k = male[male['lyrics'].str.contains("기다려", na=False)]
male_k.shape

(468, 18)

In [ ]:
import multiprocessing

In [31]:
from collections import Counter

# 정규표현식을 이용하여 어떤 어미가 쓰였는지 알아보자

In [108]:
# 정규표현식을 이용하여 어떤 어미가 쓰였는지 알아보는 함수

def end_of_word(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    return result

In [142]:
# 특정 어미(search_word)의 성별 빈도수 보는 함수(단어 수)

def word_in_song2(search_word):
    r = re.compile(search_word)
    song_r = list(filter(r.match, song_t))
    female_r = list(filter(r.match, female_t))
    male_r = list(filter(r.match, male_t))
    
    song_r = format((len(song_r)/2244770)*1000, '.2f')
    female_r = format((len(female_r)/769211)*1000, '.2f')
    male_r = format((len(male_r)/1409422)*1000, '.2f')
   
    return search_word, song_r, female_r, male_r

In [113]:
# 특정 어미search_word의 성별 빈도수 보는 함수(곡 수)

def word_in_song(search_word):
    song_w = song[song['lyrics'].str.contains(search_word, na=False)]
    female_w = female[female['lyrics'].str.contains(search_word, na=False)]
    male_w = male[male['lyrics'].str.contains(search_word, na=False)]
    
    song_w = format((len(song_w)/9566)*100, '.2f')
    female_w = format((len(female_w)/3338)*100, '.2f')
    male_w = format((len(male_w)/5869)*100, '.2f')
   
    return search_word, song_w, female_w, male_w

In [139]:
r = re.compile(r".주세요$")
result = list(filter(r.match, male_t))

In [143]:
word_in_song2("주세요$")

('주세요$', '0.05', '0.10', '0.02')

In [148]:
word_in_song('beautiful')

('beautiful', '1.89', '1.26', '2.30')

In [115]:
# 어미를 보고 성별 빈도수도 함께 보는 함수
# *단어가 등장한 곡 수* 입니다
# 주의해야 할 점: $가 들어가는 것은 이걸로 함께 볼 수 없다. 위 두 함수로 따로 봐야(성별 빈도수 볼 때는 $ 빼줘야 함. 토크나이즈되지 않았으므로)

def end_of_word2(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    result2 = word_in_song(search_word)
    return result, result2

In [144]:
# 어미를 보고 성별 빈도수도 함께 보는 함수
# *총 단어 빈도수*입니다 곡 수 아님

def end_of_word3(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    result2 = word_in_song2(search_word)
    return result, result2

In [145]:
# 단어 빈도수와 곡 수 같이 보기
# 주의해야 할 점: 역시 $가 들어가는 것은 이걸로 함께 볼 수 없다. word_in_song, end_of_word3 두 함수로 따로 봐야
#(word_in_song(곡 수) 볼 때는 $ 빼줘야 함. 토크나이즈되지 않았으므로)

def end_of_word4(tokenized_list, search_word):
    r = re.compile(search_word)
    result = list(filter(r.match, tokenized_list))
    result = Counter(result).most_common(20)
    result2 = word_in_song(search_word)
    result3 = word_in_song2(search_word)
    return result, result2, result3

### 빈도수는 한 어미의 빈도수를 볼 때 유용합니다.  
r"해(줘|줄|주|요)." 처럼 어떤 어미들이 오는지 볼 때 보다  
r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일)까$" 처럼 한 어미를 볼 때  

In [146]:
end_of_word4(female_t, r"해(줘|줄|주|요).")

/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yunkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


([('해줘요', 44),
  ('해줄게', 31),
  ('해주고', 19),
  ('해줄래', 18),
  ('해주세요', 15),
  ('해주는', 10),
  ('해주고픈', 8),
  ('해줄까', 8),
  ('해줄거야', 8),
  ('해주면', 6),
  ('해주오', 6),
  ('해줄래요', 6),
  ('해줄께', 5),
  ('해주길', 5),
  ('해주던', 4),
  ('해주기', 3),
  ('해줘도', 3),
  ('해주지', 3),
  ('해주고만', 3),
  ('해줘야', 2)],
 ('해(줘|줄|주|요).', '27.18', '29.69', '25.63'),
 ('해(줘|줄|주|요).', '0.31', '0.30', '0.31'))

In [98]:
end_of_word(female_t, r"해(줘|줄|주|요).")

[('해줘요', 44),
 ('해줄게', 31),
 ('해주고', 19),
 ('해줄래', 18),
 ('해주세요', 15),
 ('해주는', 10),
 ('해주고픈', 8),
 ('해줄까', 8),
 ('해줄거야', 8),
 ('해주면', 6),
 ('해주오', 6),
 ('해줄래요', 6),
 ('해줄께', 5),
 ('해주길', 5),
 ('해주던', 4),
 ('해주기', 3),
 ('해줘도', 3),
 ('해주지', 3),
 ('해주고만', 3),
 ('해줘야', 2)]

In [97]:
end_of_word(male_t, r"해(줘|줄|주|요).")

[('해줄게', 121),
 ('해주고', 54),
 ('해줄래', 31),
 ('해줘요', 26),
 ('해주는', 22),
 ('해줘서', 17),
 ('해주지', 12),
 ('해주면', 12),
 ('해줄께', 12),
 ('해주길', 7),
 ('해줄거야', 7),
 ('해주겠니', 6),
 ('해줘야만', 6),
 ('해요말고', 6),
 ('해줘야', 5),
 ('해주던', 4),
 ('해주오', 4),
 ('해줄수', 4),
 ('해줄꺼야', 4),
 ('해주세요', 4)]

In [100]:
end_of_word(female_t, r"(돼|되)(줘|줄|어|잖|는|겠|었).")

[('되는걸', 25),
 ('되는데', 22),
 ('돼줄게', 16),
 ('되겠지', 15),
 ('되어준', 13),
 ('되어줘', 12),
 ('되줄게', 12),
 ('되겠니', 10),
 ('되어서', 10),
 ('되었어', 9),
 ('되는거야', 9),
 ('되겠어', 8),
 ('되는지', 8),
 ('되었죠', 8),
 ('돼줄래', 8),
 ('되잖아', 7),
 ('돼줘요', 6),
 ('되어줄', 6),
 ('되어주오', 6),
 ('되어줄게', 5)]

In [101]:
end_of_word(male_t, r"(돼|되)(줘|줄|어|잖|는|겠|었).")

[('되어줘', 53),
 ('되잖아', 51),
 ('되는데', 50),
 ('돼줄게', 41),
 ('되는걸', 39),
 ('되겠어', 37),
 ('되었어', 34),
 ('되겠지', 32),
 ('되어서', 31),
 ('되어줄게', 27),
 ('되겠니', 24),
 ('되는지', 21),
 ('되어준', 19),
 ('되어도', 16),
 ('되었지', 14),
 ('되었죠', 14),
 ('돼줄래', 13),
 ('되어버린', 13),
 ('되는거야', 11),
 ('되었던', 10)]

In [117]:
end_of_word(female_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일|하|보|가)(래|자|까)$")

[('아닐까', 46),
 ('말할까', 44),
 ('해볼래', 36),
 ('사랑할래', 29),
 ('멀어질까', 27),
 ('해볼까', 26),
 ('말할래', 25),
 ('보일까', 22),
 ('어떡할래', 22),
 ('이럴까', 21),
 ('가질래', 20),
 ('바라볼래', 17),
 ('안할래', 17),
 ('전할래', 16),
 ('어떨까', 14),
 ('시작할래', 14),
 ('사랑일까', 13),
 ('말해볼까', 12),
 ('다가갈래', 11),
 ('꿈일까', 11)]

In [105]:
end_of_word(male_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일|하|보|가)(래|자|까)$")

[('아닐까', 47),
 ('어떨까', 45),
 ('이럴까', 44),
 ('보일까', 35),
 ('말할까', 29),
 ('가보자', 28),
 ('가질래', 23),
 ('해볼까', 22),
 ('사라질까', 22),
 ('말할래', 20),
 ('돌아갈래', 20),
 ('나가자', 18),
 ('사랑일까', 18),
 ('삐뚤어질래', 18),
 ('놀아보자', 17),
 ('왜일까', 16),
 ('잊혀질까', 16),
 ('무엇일까', 15),
 ('걸어가자', 14),
 ('사랑할래', 14)]

In [59]:
end_of_word(female_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일)까$")

[('아닐까', 46),
 ('말할까', 44),
 ('멀어질까', 27),
 ('해볼까', 26),
 ('보일까', 22),
 ('이럴까', 21),
 ('어떨까', 14),
 ('사랑일까', 13),
 ('말해볼까', 12),
 ('꿈일까', 11),
 ('걸어볼까', 10),
 ('고백해볼까', 10),
 ('그럴까', 10),
 ('시작할까', 9),
 ('떠나볼까', 8),
 ('다가가볼까', 8),
 ('좋아할까', 8),
 ('왜일까', 8),
 ('착각일까', 7),
 ('시작해볼까', 7)]

In [60]:
end_of_word(male_t, r".{1,4}(할|볼|갈|걸|말|뭘|될|갈|놀|알|럴|떨|질|닐|볼|일)까$")

[('아닐까', 47),
 ('어떨까', 45),
 ('이럴까', 44),
 ('보일까', 35),
 ('말할까', 29),
 ('해볼까', 22),
 ('사라질까', 22),
 ('사랑일까', 18),
 ('왜일까', 16),
 ('잊혀질까', 16),
 ('무엇일까', 15),
 ('욕심일까', 14),
 ('멀어질까', 14),
 ('시작해볼까', 12),
 ('가볼까', 12),
 ('걸어볼까', 11),
 ('전해질까', 11),
 ('변해볼까', 10),
 ('그럴까', 9),
 ('너일까', 9)]

In [62]:
end_of_word(female_t, r".{1,4}(할|볼|갈|말|될|갈|놀|럴|볼)래$")

[('해볼래', 36),
 ('사랑할래', 29),
 ('말할래', 25),
 ('어떡할래', 22),
 ('바라볼래', 17),
 ('안할래', 17),
 ('전할래', 16),
 ('시작할래', 14),
 ('다가갈래', 11),
 ('그만할래', 10),
 ('고백할래', 10),
 ('들어볼래', 8),
 ('놀아볼래', 8),
 ('안아볼래', 7),
 ('유혹할래', 7),
 ('가볼래', 7),
 ('살아갈래', 7),
 ('날아갈래', 6),
 ('즐겨볼래', 6),
 ('뛰어볼래', 6)]

In [63]:
end_of_word(male_t, r".{1,4}(할|볼|갈|말|될|갈|놀|럴|볼)래$")

[('말할래', 20),
 ('돌아갈래', 20),
 ('사랑할래', 14),
 ('바라볼래', 10),
 ('믿어볼래', 9),
 ('다가갈래', 8),
 ('해볼래', 8),
 ('고백할래', 8),
 ('날아갈래', 8),
 ('가볼래', 6),
 ('살아갈래', 6),
 ('그만할래', 6),
 ('걸어볼래', 6),
 ('뭐할래', 6),
 ('데려갈래', 5),
 ('걸어갈래', 5),
 ('지켜볼래', 5),
 ('말해볼래', 5),
 ('노래할래', 5),
 ('떠나볼래', 5)]

In [102]:
end_of_word(female_t, r".{1,3}(하|보|가|놀)자$")

[('그만하자', 7),
 ('달려보자', 6),
 ('살아보자', 6),
 ('노래하자', 4),
 ('뛰어보자', 4),
 ('떠나보자', 4),
 ('나가자', 4),
 ('함께하자', 3),
 ('생각하자', 3),
 ('올라가자', 2),
 ('말해보자', 2),
 ('아파하자', 2),
 ('해보자', 2),
 ('뛰어가자', 2),
 ('어디보자', 2),
 ('두고보자', 2),
 ('설쳐보자', 2),
 ('영원하자', 2),
 ('날아가자', 2),
 ('믿어보자', 2)]

In [103]:
end_of_word(male_t, r".{1,3}(하|보|가|놀)자$")

[('가보자', 28),
 ('나가자', 18),
 ('놀아보자', 17),
 ('걸어가자', 14),
 ('달려보자', 13),
 ('안아보자', 13),
 ('시작하자', 12),
 ('그만하자', 12),
 ('돌아가자', 9),
 ('행복하자', 7),
 ('해보자', 6),
 ('사랑하자', 6),
 ('날아가자', 6),
 ('영원하자', 6),
 ('외워보자', 6),
 ('함께하자', 5),
 ('뛰어보자', 5),
 ('노래하자', 5),
 ('떠나가자', 4),
 ('도전하자', 4)]

In [67]:
end_of_word(female_t, r".{1,4}거야")

[('줄거야', 37),
 ('할거야', 37),
 ('될거야', 35),
 ('않을거야', 28),
 ('갈거야', 25),
 ('없을거야', 22),
 ('온거야', 22),
 ('사랑할거야', 20),
 ('된거야', 19),
 ('하는거야', 18),
 ('올거야', 16),
 ('있을거야', 16),
 ('한거야', 13),
 ('가는거야', 13),
 ('있는거야', 12),
 ('기다릴거야', 12),
 ('없는거야', 11),
 ('말할거야', 11),
 ('되는거야', 9),
 ('말거야', 9)]

In [66]:
end_of_word(male_t, r".{1,4}거야")

[('않을거야', 73),
 ('할거야', 58),
 ('없을거야', 57),
 ('없는거야', 51),
 ('될거야', 39),
 ('있을거야', 37),
 ('있는거야', 36),
 ('된거야', 35),
 ('줄거야', 33),
 ('사랑할거야', 27),
 ('가는거야', 26),
 ('하는거야', 25),
 ('온거야', 25),
 ('말할거야', 23),
 ('말거야', 23),
 ('지켜줄거야', 21),
 ('만들거야', 19),
 ('갈거야', 18),
 ('모를거야', 17),
 ('위한거야', 15)]

In [68]:
end_of_word(female_t, r".{1,4}요$")

[('말아요', 198),
 ('봐요', 190),
 ('필요', 180),
 ('마요', 169),
 ('사랑해요', 150),
 ('있어요', 137),
 ('해요', 129),
 ('와요', 126),
 ('몰라요', 122),
 ('없어요', 114),
 ('있나요', 107),
 ('좋아요', 104),
 ('줘요', 101),
 ('싶어요', 90),
 ('않아요', 90),
 ('안아줘요', 84),
 ('해봐요', 80),
 ('주세요', 79),
 ('없나요', 73),
 ('말해줘요', 71)]

In [73]:
end_of_word(male_t, r".{1,4}요$")

[('필요', 333),
 ('말아요', 327),
 ('마요', 306),
 ('사랑해요', 243),
 ('봐요', 192),
 ('해요', 185),
 ('있어요', 180),
 ('있나요', 174),
 ('없어요', 133),
 ('줘요', 97),
 ('가요', 96),
 ('그래요', 94),
 ('않아요', 87),
 ('알아요', 87),
 ('미안해요', 80),
 ('믿어요', 73),
 ('싶어요', 72),
 ('같아요', 71),
 ('돼요', 66),
 ('여보세요', 63)]

In [75]:
end_of_word(female_t, r".{1,4}줄게$")

[('보여줄게', 95),
 ('해줄게', 31),
 ('지켜줄게', 26),
 ('말해줄게', 22),
 ('안아줄게', 17),
 ('보내줄게', 17),
 ('돼줄게', 16),
 ('비춰줄게', 14),
 ('되줄게', 12),
 ('들어줄게', 12),
 ('들려줄게', 11),
 ('채워줄게', 8),
 ('지워줄게', 7),
 ('잊어줄게', 7),
 ('기다려줄게', 7),
 ('감싸줄게', 6),
 ('잡아줄게', 6),
 ('전해줄게', 6),
 ('되어줄게', 5),
 ('아껴줄게', 5)]

In [74]:
end_of_word(male_t, r".{1,4}줄게$")

[('지켜줄게', 196),
 ('보여줄게', 148),
 ('해줄게', 121),
 ('안아줄게', 71),
 ('돼줄게', 41),
 ('있어줄게', 41),
 ('보내줄게', 30),
 ('되어줄게', 27),
 ('아껴줄게', 22),
 ('비춰줄게', 18),
 ('말해줄게', 17),
 ('지워줄게', 17),
 ('불러줄게', 16),
 ('받아줄게', 15),
 ('잘해줄게', 13),
 ('빌어줄게', 12),
 ('알려줄게', 12),
 ('채워줄게', 12),
 ('감싸줄게', 11),
 ('다해줄게', 11)]

In [95]:
end_of_word(female_t, r".{1,4}줘요$")

[('안아줘요', 84),
 ('말해줘요', 71),
 ('해줘요', 44),
 ('말아줘요', 19),
 ('와줘요', 18),
 ('잡아줘요', 17),
 ('봐줘요', 16),
 ('있어줘요', 15),
 ('보여줘요', 14),
 ('사랑해줘요', 13),
 ('기억해줘요', 11),
 ('웃어줘요', 11),
 ('알아줘요', 9),
 ('지켜줘요', 9),
 ('위로해줘요', 9),
 ('꺼줘요', 9),
 ('속삭여줘요', 7),
 ('구해줘요', 7),
 ('바라봐줘요', 7),
 ('전해줘요', 7)]

In [93]:
end_of_word(male_t, r".{1,4}줘요$")

[('사랑해줘요', 30),
 ('불러줘요', 29),
 ('안아줘요', 28),
 ('해줘요', 26),
 ('있어줘요', 25),
 ('돌아와줘요', 22),
 ('말해줘요', 15),
 ('말아줘요', 13),
 ('기억해줘요', 13),
 ('믿어줘요', 12),
 ('들어줘요', 8),
 ('되어줘요', 7),
 ('알아줘요', 7),
 ('웃어줘요', 7),
 ('받아줘요', 7),
 ('약속해줘요', 7),
 ('용서해줘요', 6),
 ('잡아줘요', 6),
 ('와줘요', 5),
 ('기다려줘요', 5)]

In [86]:
end_of_word(female_t, r".{1,3}세요$")

[('주세요', 79),
 ('해주세요', 15),
 ('안아주세요', 10),
 ('여보세요', 9),
 ('받아주세요', 7),
 ('보세요', 6),
 ('오세요', 6),
 ('놔두세요', 6),
 ('안녕하세요', 6),
 ('하세요', 5),
 ('말해주세요', 5),
 ('찾아오세요', 5),
 ('생각하세요', 4),
 ('지켜내세요', 4),
 ('행복하세요', 4),
 ('느껴보세요', 3),
 ('알려주세요', 3),
 ('비춰주세요', 3),
 ('마세요', 3),
 ('비켜나세요', 2)]

In [84]:
end_of_word(male_t, r".{1,4}세요$")

[('여보세요', 63),
 ('안녕하세요', 46),
 ('마세요', 23),
 ('주세요', 22),
 ('누구세요', 13),
 ('가세요', 13),
 ('가지마세요', 7),
 ('하세요', 7),
 ('행복하세요', 7),
 ('받아주세요', 5),
 ('말해보세요', 5),
 ('일어나세요', 4),
 ('이겨내세요', 4),
 ('느껴보세요', 4),
 ('보세요', 4),
 ('해주세요', 4),
 ('어디세요', 4),
 ('보여주세요', 3),
 ('아껴주세요', 3),
 ('안아주세요', 3)]

In [88]:
end_of_word(female_t, r".{1,3}주세요$")

[('해주세요', 15),
 ('안아주세요', 10),
 ('받아주세요', 7),
 ('말해주세요', 5),
 ('알려주세요', 3),
 ('비춰주세요', 3),
 ('잡아주세요', 2),
 ('알아주세요', 2),
 ('불러주세요', 2),
 ('아껴주세요', 2),
 ('지켜주세요', 2),
 ('만나주세요', 2),
 ('보내주세요', 2),
 ('기억해주세요', 1),
 ('쳐주세요', 1),
 ('위로해주세요', 1),
 ('써주세요', 1),
 ('짜주세요', 1),
 ('도와주세요', 1),
 ('이뤄주세요', 1)]

In [87]:
end_of_word(male_t, r".{1,3}주세요$")

[('받아주세요', 5),
 ('해주세요', 4),
 ('보여주세요', 3),
 ('아껴주세요', 3),
 ('안아주세요', 3),
 ('녹여주세요', 2),
 ('말해주세요', 2),
 ('물어봐주세요', 2),
 ('있어주세요', 2),
 ('용서해주세요', 2),
 ('이해해주세요', 2),
 ('돌려주세요', 2),
 ('대답해주세요', 1),
 ('살아주세요', 1),
 ('잡아주세요', 1),
 ('외처주세요', 1),
 ('춰주세요', 1),
 ('기억해주세요', 1),
 ('불러주세요', 1),
 ('갖다주세요', 1)]

In [89]:
end_of_word(female_t, r".지마$")

[('하지마', 338),
 ('가지마', 100),
 ('보지마', 36),
 ('잊지마', 29),
 ('찾지마', 26),
 ('놓지마', 26),
 ('웃지마', 14),
 ('묻지마', 14),
 ('굴지마', 13),
 ('두지마', 13),
 ('잡지마', 13),
 ('오지마', 13),
 ('울지마', 11),
 ('쓰지마', 9),
 ('팔지마', 8),
 ('듣지마', 8),
 ('빼지마', 7),
 ('속지마', 7),
 ('먹지마', 7),
 ('붙지마', 7)]

In [90]:
end_of_word(male_t, r".지마$")

[('하지마', 522),
 ('가지마', 182),
 ('잊지마', 76),
 ('울지마', 76),
 ('보지마', 56),
 ('찾지마', 29),
 ('놓지마', 27),
 ('막지마', 22),
 ('입지마', 18),
 ('쓰지마', 17),
 ('살지마', 17),
 ('두지마', 16),
 ('묻지마', 13),
 ('있지마', 12),
 ('믿지마', 12),
 ('주지마', 10),
 ('잡지마', 9),
 ('웃지마', 9),
 ('숨지마', 8),
 ('쫄지마', 8)]

In [92]:
end_of_word(female_t, r".{1,4}지마$")

[('하지마', 338),
 ('가지마', 100),
 ('떠나지마', 59),
 ('보지마', 36),
 ('망설이지마', 31),
 ('떠나가지마', 31),
 ('잊지마', 29),
 ('멈추지마', 27),
 ('말하지마', 26),
 ('찾지마', 26),
 ('놓지마', 26),
 ('걱정하지마', 21),
 ('울리지마', 20),
 ('서두르지마', 19),
 ('행복하지마', 17),
 ('바라보지마', 16),
 ('착각하지마', 15),
 ('이러지마', 15),
 ('웃지마', 14),
 ('묻지마', 14)]

In [91]:
end_of_word(male_t, r".{1,4}지마$")

[('하지마', 522),
 ('가지마', 182),
 ('떠나지마', 115),
 ('잊지마', 76),
 ('울지마', 76),
 ('떠나가지마', 73),
 ('걱정하지마', 58),
 ('보지마', 56),
 ('멈추지마', 52),
 ('말하지마', 47),
 ('이러지마', 46),
 ('까불지마', 35),
 ('찾지마', 29),
 ('망설이지마', 27),
 ('생각하지마', 27),
 ('놓지마', 27),
 ('막지마', 22),
 ('버리지마', 20),
 ('입지마', 18),
 ('피하지마', 17)]

In [72]:
# '필요'라는 단어는 어미가 포함되지 않은 단어이므로 제외한다.
import re

r = re.compile("^(?!.*필).*$")
newlist = list(filter(r.match, female_t)) # Read Note
Counter(newlist).most_common(20)

[('내', 9708),
 ('I', 8875),
 ('you', 7537),
 ('난', 6575),
 ('날', 5678),
 ('me', 5513),
 ('oh', 4538),
 ('t', 4415),
 ('나', 4141),
 ('Oh', 4095),
 ('더', 4047),
 ('널', 3990),
 ('love', 3821),
 ('나를', 3647),
 ('it', 3618),
 ('너', 3424),
 ('내가', 3373),
 ('수', 3290),
 ('넌', 3228),
 ('그', 3192)]

In [ ]:
# '필요'라는 단어는 어미가 포함되지 않은 단어이므로 제외한다.
import re

r = re.compile("^(?!.*필).*$")
newlist2 = list(filter(r.match, newlist2)) # Read Note
print(len(newlist2))

In [ ]:
# tf-idf로 여자 아이돌의 가사에만 많이 나오는 단어, 남자 아이돌의 가사에만 많이 나오는 단어를 보고 싶다.